https://discordpy.readthedocs.io/en/latest/discord.html

Direct for discord using javascript: https://github.com/Rei-x/discord-speech-recognition?tab=readme-ov-file

In [ ]:
!pip install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.10.0+cu102 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+rocm5.5, 2.1.1+rocm5.6, 2.1.2, 2.1.2+cpu, 2.1.2+cpu.cxx11.abi, 2.1.2+cu118, 2.1.2+cu121, 2.1.2+cu121.with.pypi.cudnn, 2.1.2+rocm5.5, 2.1.2+rocm5.6, 2.2.0, 2.2.0+cpu, 2.2.0+cpu.cxx11.abi, 2.2.0+cu118, 2.2.0+cu121, 2.2.0+rocm5.6, 2.2.

In [ ]:
#@title Check version:
import torch
import pyannote.audio
print(torch.__version__)
print(pyannote.audio.__version__)

ModuleNotFoundError: No module named 'torch'

In [ ]:
# !apt update
# !apt install libcudnn8=8.9.5.*-1+cuda11.8

In [ ]:
# !pip3 install whisperx
# !apt install ffmpeg -y

In [ ]:
# !pip install git+https://github.com/m-bain/whisperx.git --upgrade

##<font color='gold'>Step 1: Setup

I still have to research whether shisper is SOTA and commercial licence

Here is "whisperx" fo rexample: https://www.youtube.com/watch?v=SAIsk0i7KgU

https://discordpy.readthedocs.io/en/latest/discord.html

In [ ]:
#@title Setup definitions for the script
# Define some ANSI escape codes for colors
GREEN = "\033[92m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
RESET = "\033[0m"
MAGENTA = '\033[35m'
RED = '\033[31m'
ORANGE = "\033[38;5;208m"
RESET = '\033[0m'

def get_bot_status():
  if 'bot_task' in globals() and not bot_task.done():
      print("Bot is RUNNING. 🔴 ")
  else:
      print("Bot is NOT (!) running. 🛑")

get_bot_status()

Bot is NOT (!) running. 🛑


In [ ]:
#@title Install requirements
%%capture
'''
This is more cumbersome than necessary, but
this way it creates a requirements.txt
which is desirable.
'''

content = """py-cord
python-dotenv"""
# openai-whisper""" # openai-whisper is slow to install...
# openai-whisper is not required if working with whisperx


with open("requirements.txt", "w") as file:
    file.write(content)

print("requirements.txt has been created.")

!pip install -r ./requirements.txt
!pip install PyNaCl
!pip install nest_asyncio


%cd /content/
!rm -rf /content/tiny-tutorials
!git clone https://github.com/runfish5/tiny-tutorials.git

%cd /content/tiny-tutorials/discord-transcription-bot

In [ ]:
#@title pip install whisperx
%%capture
"""
This is the transcription AI engine.
Takes some time.
"""
try:
    import whisperx
except ImportError:
    # !pip install whisperx
    !pip install git+https://github.com/m-bain/whisperx.git --upgrade
    import whisperx


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


In [ ]:
# device = "cuda"
# audio_file = "audio.mp3"
# batch_size = 16 # reduce if low on GPU mem
# compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# model = whisperx.load_model("large-v2", device, compute_type=compute_type)


In [ ]:
#@title import packages; Load environment variables
import asyncio
import nest_asyncio

nest_asyncio.apply()

from pprint import pprint
from dotenv import load_dotenv
import discord
import tempfile
import os

env_content = """DISCORD_BOT_TOKEN=<alpha-numeric key here>
"""
with open(".env", "w") as env_file:
    env_file.write(env_content)

load_dotenv()
print("'.env' file created and loaded successfully!")

'.env' file created and loaded successfully!


In [ ]:
#@title set your parameters
# Whisper is on large-v2 by default
model_size = 'large-v2' #@param ["tiny", "small", "medium", "large-v2", "tiny.en", "small.en", "medium.en"]


device = 'cuda' #@param ["cpu", "cuda"]

audio_file_language = 'en' #@param ["en"]


In [ ]:
#@title set the AI model
import whisperx
import gc
import model_config
import torch


batch_size = 16 # reduce if low on GPU mem
if device == 'cpu':
  compute_type = 'auto'
else:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)


# 1. Transcribe with original whisper (batched)
model_config.model = whisperx.load_model(model_size, device, compute_type=compute_type)

model_a, metadata_model_a = whisperx.load_align_model(language_code=audio_file_language,#result["language"],
                                              device=device)


# import whisper
# model_config.model = whisper.load_model("base")


print(metadata_model_a)
print("\nModel assigned to configuration")


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.
{'language': 'en', 'dictionary': {'-': 0, '|': 1, 'e': 2, 't': 3, 'a': 4, 'o': 5, 'n': 6, 'i': 7, 'h': 8, 's': 9, 'r': 10, 'd': 11, 'l': 12, 'u': 13, 'm': 14, 'w': 15, 'c': 16, 'f': 17, 'g': 18, 'y': 19, 'p': 20, 'b': 21, 'v': 22, 'k': 23, "'": 24, 'x': 25, 'j': 26, 'q': 27, 'z': 28}, 'type': 'torchaudio'}

Model assigned to configuration


##<font color='gold'>Step 2: Turn on bot

In [ ]:
#@title Get bot status  (🔴/🛑)
get_bot_status()


Bot is NOT (!) running. 🛑


In [ ]:
#@title Execute main.py
exec(open("main.py").read())

# Global stop flag
running = True

async def keep_alive():
    global running
    while running:
        await asyncio.sleep(1)  # Keep the loop alive without blocking

# Start the bot and the keep-alive task
loop = asyncio.get_event_loop()
bot_task = loop.create_task(run_bot())
keep_alive_task = loop.create_task(keep_alive())

get_bot_status()

This script is meant to be run from the notebook.
Bot is RUNNING. 🔴 


In [ ]:
#@title  close the bot

await bot.close()

get_bot_status()
'''
Set the stop flag
'''
running = False

Bot is NOT (!) running. 🛑


In [ ]:
'''
For downloads we zip it.
(Google Colab doesn't support download and upload of directories.)
'''
# Get current date in yymmdd format
current_date = datetime.now().strftime("%y%m%d")

# Create base zip filename with date prefix
base_zip_filename = f"{current_date}_recordings.zip"

# Check if file exists and add version number if needed
i = 1
zip_filename = base_zip_filename
while os.path.exists(zip_filename):
    zip_filename = f"{current_date}_recordings_{i}.zip"
    i += 1

# Zip the recordings directory with the dated filename
!zip -r {zip_filename} recordings

print(f"Created zip file: {zip_filename}")

  adding: recordings/ (stored 0%)
  adding: recordings/runfishrun/ (stored 0%)
  adding: recordings/runfishrun/.ipynb_checkpoints/ (stored 0%)
  adding: recordings/runfishrun/metadata.json (deflated 60%)
  adding: recordings/runfishrun/session_20250329_14M58.wav (deflated 91%)
  adding: recordings/alpakatm/ (stored 0%)
  adding: recordings/alpakatm/metadata.json (deflated 57%)
  adding: recordings/alpakatm/session_20250329_14M58.wav (deflated 72%)
  adding: recordings/lu_xo/ (stored 0%)
  adding: recordings/lu_xo/metadata.json (deflated 57%)
  adding: recordings/lu_xo/session_20250329_14M58.wav (deflated 86%)


##<font color='gold'>Step 3: Transcribe Audio Files

In [ ]:
#@title not
# import gdown
# modul_link = "https://drive.google.com/file/d/1tEu6xqF9DMQ9FEcXZ4guG3uymxiVEtcZ/view?usp=sharing" # UTIL.ipynb
# output = '/content/recordings.zip'

# gdown.download(modul_link, output, quiet=False)

# # !jupyter nbconvert --to script sb_util.ipynb --output sb_util.py
# # !mkdir -p project_utilities

# # !mv 'sb_util.py.txt' 'sb_util.py'
# # !mv sb_util.py project_utilities/
# # !rm sb_util.ipynb
# # sys.path.append('project_utilities')
# # import sb_util

# # importlib.reload(sb_util)

# # #Testing
# # print(sb_util.col_data_set)


In [ ]:
# !gdown "https://drive.google.com/file/d/1tEu6xqF9DMQ9FEcXZ4guG3uymxiVEtcZ/view?usp=sharing"

In [ ]:
#@title gdown
import gdown

# Replace with your shared link
shared_link = "https://drive.google.com/file/d/1tEu6xqF9DMQ9FEcXZ4guG3uymxiVEtcZ/view?usp=sharing"

# Extract the file ID from the shared link
file_id = shared_link.split("/")[5]

# Download the file
output_path = "/content/downloaded_file.zip"

gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
import zipfile

# Unzip the file
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall("/content/extracted_files")

Downloading...
From (original): https://drive.google.com/uc?id=1tEu6xqF9DMQ9FEcXZ4guG3uymxiVEtcZ
From (redirected): https://drive.google.com/uc?id=1tEu6xqF9DMQ9FEcXZ4guG3uymxiVEtcZ&confirm=t&uuid=09d6b3ea-d9ba-450a-9981-91c7f3f6565d
To: /content/downloaded_file.zip
100%|██████████| 169M/169M [00:03<00:00, 43.0MB/s]


In [ ]:
#@title <font color='magenta'>COPY THE RECORDINGS (.wav)
# !mv /content/extracted_files/recordings .
!cp -r /content/extracted_files/recordings .


In [ ]:
#@title transcribe_all_recordings (timeouted)
# import os
# import json
# import model_config  # Assuming this module defines the Whisper model

# def transcribe_all_recordings():
#     """
#     Transcribes all audio recordings in the 'recordings' directory using the Whisper model.
#     Saves transcriptions as text files and updates metadata.json for each user.
#     """
#     # Set the Whisper model
#     model_config.model = model_config.model  # Ensure model is set (redundant here if already set)

#     # Define the root directory for recordings
#     recordings_dir = "recordings"

#     # Check if the recordings directory exists
#     if not os.path.exists(recordings_dir):
#         print("Error: The 'recordings' directory does not exist.")
#         return

#     # Iterate through each user's directory in recordings
#     for user_id in os.listdir(recordings_dir):
#         user_dir = os.path.join(recordings_dir, user_id)

#         # Skip if it's not a directory
#         if not os.path.isdir(user_dir):
#             continue

#         # Path to metadata.json for this user
#         metadata_path = os.path.join(user_dir, "metadata.json")

#         # Check if metadata.json exists
#         if not os.path.exists(metadata_path):
#             print(f"Warning: No metadata.json found for user {user_id}. Skipping.")
#             continue

#         # Load the metadata
#         with open(metadata_path, "r") as f:
#             metadata = json.load(f)

#         # Process each recording in the metadata
#         for recording in metadata.get("recordings", []):
#             audio_file = recording.get("audio_file")

#             # Skip if no audio file is specified
#             if not audio_file:
#                 print(f"Warning: No audio_file specified in metadata for user {user_id}. Skipping entry.")
#                 continue

#             audio_path = os.path.join(user_dir, audio_file)

#             # Check if the audio file exists
#             if not os.path.exists(audio_path):
#                 print(f"Warning: Audio file {audio_path} not found. Skipping.")
#                 continue

#             # Transcribe the audio using the Whisper model
#             try:
#                 audio = whisperx.load_audio(audio_path)
#                 transcription = model_config.model.transcribe(audio)
#                 print(f">>  {transcription}")

#                 print(f'{YELLOW}audio loaded{RESET}')
#                 transcription = whisperx.align(transcription["segments"],
#                                                model_a,
#                                                metadata_model_a,
#                                                audio,
#                                                device,
#                                                return_char_alignments=False)

#                 print(f'{YELLOW}align achieved{RESET}')

#                 if transcription.get('segments', None): # means its the whisperx model and not whisper.
#                 #     print(f'{GREEN}WHISPERX recognized{RESET}')
#                 #     transcription_text = "TTTTTTTTTT".join(
#                 #         [text['text'] for text in transcription["segments"]]
#                 #     )
#                     transcription_text = transcription
#                 else:
#                     transcription_text = transcription["text"]
#             except Exception as e:
#                 print(f"Error: Failed to transcribe {audio_path}: {str(e)}")
#                 continue

#             # Define the transcription file name and path
#             transcription_file = f"{os.path.splitext(audio_file)[0]}_transcription.txt"
#             transcription_path = os.path.join(user_dir, transcription_file)

#             # Save the transcription to a text file
#             with open(transcription_path, "w") as f:
#                 # f.write(transcription_text)
#                 f.write(
#                     json.dumps(transcription_text, indent=4, sort_keys=False, ensure_ascii=False )
#                 )

#             # Update the recording metadata with the transcription file path
#             recording["transcription_file"] = transcription_file

#         # Save the updated metadata back to metadata.json
#         with open(metadata_path, "w") as f:
#             json.dump(metadata, f, indent=4)

#         print(f"Success: Transcriptions completed for user {user_id}")


# transcribe_all_recordings()

https://x.com/i/grok/share/EtI0QCIeDVrgkE0lx5dGeaNS9

In [ ]:
# !pip install whisperx
import whisperx
# model_size ='large-v2'
model_size ='medium'
device = "cuda"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)
# compute_type = "float32"

audio_path = '/content/extracted_files/recordings/alpakatm/session_20250329_14M58.wav'

# transcription = model_config.model.transcribe(audio)

model = whisperx.load_model(model_size, device, compute_type=compute_type)

audio = whisperx.load_audio(audio_path)
transcription = model.transcribe(audio, batch_size=batch_size)

print("Transcription done")

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


/usr/local/lib/python3.11/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


In [ ]:
!whisperx examples/sample01.wav

2025-04-18 09:51:34.678279: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 09:51:34.696177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744969894.717926    2338 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744969894.724557    2338 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 09:51:34.746451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
#@title grok?
try:
    print(f"Processing audio: {audio_path}")
    audio = whisperx.load_audio(audio_path)
    transcription = model_config.model.transcribe(audio)
    print(f"Transcription completed: {audio_path}")
    # Skip alignment for testing
    # transcription = whisperx.align(...)
    transcription_text = transcription["text"]
except Exception as e:
    print(f"Error: Failed to transcribe {audio_path}: {str(e)}")
    continue

In [ ]:
#@title grok?
audio_path = os.path.join(user_dir, audio_file)
# =====

import whisperx
audio_path = "path/to/problematic/audio.wav"
audio = whisperx.load_audio(audio_path)
print("Audio loaded successfully")


try:
    print(f"Processing audio: {audio_path}")
    audio = whisperx.load_audio(audio_path)
    print(f"Audio loaded: {audio_path}")
    transcription = model_config.model.transcribe(audio)
    print(f"Transcription completed: {audio_path}")
    transcription = whisperx.align(transcription["segments"],
                                  model_a,
                                  metadata_model_a,
                                  audio,
                                  device,
                                  return_char_alignments=False)
    print(f"Alignment completed: {audio_path}")
    transcription_text = transcription if transcription.get('segments', None) else transcription["text"]
except Exception as e:
    print(f"Error: Failed to transcribe {audio_path}: {str(e)}")
    # continue

NameError: name 'audio_path' is not defined

In [ ]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [ ]:
import os
import json
import time
import model_config  # Assuming this module defines the Whisper model

def transcribe_all_recordings(skip_existing=True):
    """
    Transcribes all audio recordings in the 'recordings' directory using the Whisper model.
    Saves transcriptions as text files and updates metadata.json for each user.
    If skip_existing is True, it will not re-transcribe already existing transcription files.
    """
    model_config.model = model_config.model  # Ensure model is set (redundant here if already set)

    recordings_dir = "recordings"
    if not os.path.exists(recordings_dir):
        print("Error: The 'recordings' directory does not exist.")
        return

    for user_id in os.listdir(recordings_dir):
        user_dir = os.path.join(recordings_dir, user_id)
        if not os.path.isdir(user_dir):
            continue

        metadata_path = os.path.join(user_dir, "metadata.json")
        if not os.path.exists(metadata_path):
            print(f"Warning: No metadata.json found for user {user_id}. Skipping.")
            continue

        with open(metadata_path, "r") as f:
            metadata = json.load(f)

        for recording in metadata.get("recordings", []):
            audio_file = recording.get("audio_file")
            if not audio_file:
                print(f"Warning: No audio_file specified in metadata for user {user_id}. Skipping entry.")
                continue

            audio_path = os.path.join(user_dir, audio_file)
            if not os.path.exists(audio_path):
                print(f"Warning: Audio file {audio_path} not found. Skipping.")
                continue

            transcription_file = f"{os.path.splitext(audio_file)[0]}_transcription.txt"
            transcription_path = os.path.join(user_dir, transcription_file)

            if skip_existing and os.path.exists(transcription_path):
                print(f"Skipping existing transcription for {audio_file}.")
                continue

            try:
                start_time = time.time()
                audio = whisperx.load_audio(audio_path)
                transcription = model_config.model.transcribe(audio)
                transcription = whisperx.align(transcription["segments"],
                                               model_a,
                                               metadata_model_a,
                                               audio,
                                               device,
                                               return_char_alignments=False)
                transcription_text = transcription if transcription.get('segments', None) else transcription["text"]
                end_time = time.time()
                duration = end_time - start_time
            except Exception as e:
                print(f"Error: Failed to transcribe {audio_path}: {str(e)}")
                continue

            with open(transcription_path, "w") as f:
                f.write(json.dumps(transcription_text, indent=4, ensure_ascii=False))

            recording["transcription_file"] = transcription_file
            recording["transcription_time_seconds"] = round(duration, 2)
            print(f"Transcribed {audio_file} in {duration:.2f} seconds.")

        with open(metadata_path, "w") as f:
            json.dump(metadata, f, indent=4)

        print(f"Success: Transcriptions completed for user {user_id}")

# Example usage:
# transcribe_all_recordings(skip_existing=True)  # Skips existing transcriptions
transcribe_all_recordings(skip_existing=False)  # Re-transcribes all files


/usr/local/lib/python3.11/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


In [ ]:
#@title adjust_transcription_times
import json
import os
from datetime import datetime

# Define color codes for output (optional, assuming they’re defined elsewhere if needed)
GREEN = '\033[92m'
RESET = '\033[0m'

def load_json_file(file_path):
    """Loads a JSON file and returns its contents."""
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading {file_path}: {e}")
        return None

def calculate_time_offset(recording, reference_time):
    """
    Calculates time difference between user start time and the reference time in seconds.

    Args:
        recording (dict): Recording metadata with user_recording_start_time.
        reference_time (datetime): The unified reference time (earliest session_start_time).

    Returns:
        float: Time offset in seconds, or None if parsing fails.
    """
    try:
        user_dt = datetime.strptime(recording["user_recording_start_time"], "%Y-%m-%d %H:%M:%S")
        offset = (user_dt - reference_time).total_seconds()
        print(f"{GREEN}total_seconds offset for {recording['audio_file']}: {offset}{RESET}")
        return offset
    except ValueError as e:
        print(f"Time parsing failed for {recording['audio_file']}: {e}")
        return None

def get_transcription_path(user_dir, audio_file):
    """Constructs and validates the transcription file path."""
    audio_base = os.path.splitext(audio_file)[0]
    trans_file = f"{audio_base}_transcription.txt"
    trans_path = os.path.join(user_dir, trans_file)
    if not os.path.exists(trans_path):
        print(f"Missing transcription: {trans_path}")
        return None
    return trans_path

def adjust_timestamps(transcription, time_offset):
    """Adjusts all start and end times in the transcription by the offset."""
    for segment in transcription["segments"]:
        segment["start"] += time_offset
        segment["end"] += time_offset
        for word in segment["words"]:
            word["start"] = word.get("start", 0) + time_offset
            word["end"] = word.get("end", 0) + time_offset

    for word in transcription["word_segments"]:
        word["start"] = word.get("start", 0) + time_offset
        word["end"] = word.get("end", 0) + time_offset

def save_adjusted_transcription(trans_path, transcription):
    """Saves the adjusted transcription to a new file."""
    new_trans_path = trans_path.replace(".txt", "_adjusted.txt")
    with open(new_trans_path, 'w') as f:
        json.dump(transcription, f, indent=4)
    print(f"Saved adjusted transcription to {new_trans_path}")

def adjust_transcription_times(metadata_path, reference_time):
    """
    Shifts timing in transcription files to align with the earliest session start time.

    Args:
        metadata_path (str): Path to metadata JSON file (e.g., 'recordings/<user_id>/metadata.json').
        reference_time (datetime): The earliest session_start_time across all sessions.
    """
    metadata = load_json_file(metadata_path)
    if not metadata:
        return

    user_dir = os.path.dirname(metadata_path)
    for recording in metadata["recordings"]:
        time_offset = calculate_time_offset(recording, reference_time)
        if time_offset is None:
            continue

        trans_path = get_transcription_path(user_dir, recording["audio_file"])
        if not trans_path:
            continue

        transcription = load_json_file(trans_path)
        if not transcription:
            continue

        adjust_timestamps(transcription, time_offset)
        save_adjusted_transcription(trans_path, transcription)

# Process all user directories
base_dir = "/content/tiny-tutorials/discord-transcription-bot/recordings"
user_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# Step 1: Find the earliest session_start_time (T0)
session_start_times = []
for user_dir in user_dirs:
    metadata_path = os.path.join(base_dir, user_dir, "metadata.json")
    metadata = load_json_file(metadata_path)
    if metadata:
        for recording in metadata["recordings"]:
            session_start_times.append(recording["session_start_time"])

if not session_start_times:
    print("No session start times found. Cannot proceed with adjustments.")
else:
    # Convert to datetime objects and find the minimum
    session_start_datetimes = [datetime.strptime(st, "%Y-%m-%d %H:%M:%S") for st in session_start_times]
    T0 = min(session_start_datetimes)
    start_time_str = str(T0)
    print(f"Earliest session start time (T0): {T0}")

    # Step 2: Adjust transcriptions using T0 as the unified reference
    for user_dir in user_dirs:
        metadata_path = os.path.join(base_dir, user_dir, "metadata.json")
        adjust_transcription_times(metadata_path, T0)

Earliest session start time (T0): 2025-03-25 13:25:42
total_seconds offset for session_20250325_13M53.wav: 2.0
Saved adjusted transcription to /content/tiny-tutorials/discord-transcription-bot/recordings/runfishrun/session_20250325_13M53_transcription_adjusted.txt
total_seconds offset for session_20250325_13M53.wav: 2.0
Saved adjusted transcription to /content/tiny-tutorials/discord-transcription-bot/recordings/lu_xo/session_20250325_13M53_transcription_adjusted.txt


In [ ]:
# @title create_unified_transcription_object (reinstantiaed)
import os
import json
from datetime import datetime, timedelta

# Custom function to handle datetime serialization
def custom_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to an ISO 8601 string
    raise TypeError(f"Type {type(obj)} not serializable")

def create_unified_transcription_object( start_time_str, recordings_dir="recordings"):
    """
    Creates a unified JSON-like object from transcription files in the recordings directory.
    Each segment includes the username and is sorted by absolute start time.

    Args:
        recordings_dir (str): Path to the 'recordings' directory.

    Returns:
        list: Sorted list of segment dictionaries with username, start, end, and text.
    """
    all_segments = []

    # Iterate over each user directory in 'recordings'
    for user_dir in os.listdir(recordings_dir):
        user_path = os.path.join(recordings_dir, user_dir)
        if not os.path.isdir(user_path):
            continue

        # Load metadata.json
        metadata_path = os.path.join(user_path, "metadata.json")
        if not os.path.exists(metadata_path):
            print(f"Warning: metadata.json not found in {user_path}")
            continue

        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)

        # Process each recording in metadata
        for recording in metadata.get("recordings", []):
            username = recording.get("username")
            if not username:
                print(f"Warning: No username found in recording {recording}")
                continue

            if not start_time_str:
                print(f"Warning: No user_recording_start_time in {recording}")
                continue
            try:
                recording_start = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
            except ValueError as e:
                print(f"Error parsing time {start_time_str}: {e}")
                continue

            # Derive the adjusted transcription file name
            transcription_file = recording.get("transcription_file")
            if not transcription_file:
                print(f"Warning: No transcription_file in {recording}")
                continue
            # Replace '_transcription.txt' with '_transcription_adjusted.txt'
            adjusted_file = transcription_file.replace("_transcription.txt", "_transcription_adjusted.txt")
            adjusted_path = os.path.join(user_path, adjusted_file)

            # Load the transcription file
            if not os.path.exists(adjusted_path):
                print(f"Warning: {adjusted_file} not found in {user_path}")
                continue

            with open(adjusted_path, 'r', encoding='utf-8') as f:
                transcription = json.load(f)

            # Process each segment
            for segment in transcription.get("segments", []):
                rel_start = segment.get("start")
                rel_end = segment.get("end")
                text = segment.get("text")

                if rel_start is None or rel_end is None or not text:
                    print(f"Warning: Incomplete segment in {adjusted_file}: {segment}")
                    continue

                # Calculate absolute times
                abs_start = recording_start + timedelta(seconds=rel_start)
                abs_end = recording_start + timedelta(seconds=rel_end)

                # Create segment dictionary
                segment_dict = {
                    "username": username,
                    "start": abs_start,  # datetime object
                    "end": abs_end,      # datetime object
                    "text": text
                }
                all_segments.append(segment_dict)

    # Sort all segments by absolute start time
    all_segments.sort(key=lambda x: x["start"])

    return all_segments


unified_object = create_unified_transcription_object(start_time_str)
unified_obj_path = '/content/tiny-tutorials/discord-transcription-bot/recordings/unified_object.json'
final_transcript_path = '/content/tiny-tutorials/discord-transcription-bot/recordings/transcript.txt'
# Save JSON with datetime converted to string
with open(unified_obj_path, 'w') as f:
    json.dump(unified_object, f, indent=2, default=custom_serializer)

In [ ]:
#@title create_unified_transcription_object(EXP)
# import os
# import json
# from datetime import datetime, timedelta
# from typing import List, Dict, Callable

# # Custom function to handle datetime serialization
# def custom_serializer(obj):
#     if isinstance(obj, datetime):
#         return obj.isoformat()  # Convert datetime to an ISO 8601 string
#     raise TypeError(f"Type {type(obj)} not serializable")

# def sort_segments(segments: List[Dict], sort_key: str = 'start') -> List[Dict]:
#     """
#     Sort segments based on a specified key.

#     Args:
#         segments (List[Dict]): List of segment dictionaries to sort.
#         sort_key (str, optional): Key to sort by. Defaults to 'start'.
#                                   Can be 'start' or 'end'.

#     Returns:
#         List[Dict]: Sorted list of segments.

#     Raises:
#         ValueError: If an invalid sort key is provided.
#     """
#     if sort_key not in ['start', 'end']:
#         raise ValueError("Sort key must be either 'start' or 'end'")

#     return sorted(segments, key=lambda x: x[sort_key])

# def create_unified_transcription_object(
#     recordings_dir: str = "recordings",
#     sort_key: str = 'start'
# ) -> List[Dict]:
#     """
#     Creates a unified JSON-like object from transcription files in the recordings directory.
#     Each segment includes the username and can be sorted by start or end time.

#     Args:
#         recordings_dir (str): Path to the 'recordings' directory.
#         sort_key (str, optional): Key to sort segments by. Defaults to 'start'.

#     Returns:
#         list: Sorted list of segment dictionaries with username, start, end, and text.
#     """
#     all_segments = []

#     # Iterate over each user directory in 'recordings'
#     for user_dir in os.listdir(recordings_dir):
#         user_path = os.path.join(recordings_dir, user_dir)
#         if not os.path.isdir(user_path):
#             continue

#         # Load metadata.json
#         metadata_path = os.path.join(user_path, "metadata.json")
#         if not os.path.exists(metadata_path):
#             print(f"Warning: metadata.json not found in {user_path}")
#             continue

#         with open(metadata_path, 'r', encoding='utf-8') as f:
#             metadata = json.load(f)

#         # Process each recording in metadata
#         for recording in metadata.get("recordings", []):
#             username = recording.get("username")
#             if not username:
#                 print(f"Warning: No username found in recording {recording}")
#                 continue

#             # Parse the recording start time
#             start_time_str = recording.get("user_recording_start_time")
#             if not start_time_str:
#                 print(f"Warning: No user_recording_start_time in {recording}")
#                 continue
#             try:
#                 recording_start = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
#             except ValueError as e:
#                 print(f"Error parsing time {start_time_str}: {e}")
#                 continue

#             # Derive the adjusted transcription file name
#             transcription_file = recording.get("transcription_file")
#             if not transcription_file:
#                 print(f"Warning: No transcription_file in {recording}")
#                 continue
#             # Replace '_transcription.txt' with '_transcription_adjusted.txt'
#             adjusted_file = transcription_file.replace("_transcription.txt", "_transcription_adjusted.txt")
#             adjusted_path = os.path.join(user_path, adjusted_file)

#             # Load the transcription file
#             if not os.path.exists(adjusted_path):
#                 print(f"Warning: {adjusted_file} not found in {user_path}")
#                 continue

#             with open(adjusted_path, 'r', encoding='utf-8') as f:
#                 transcription = json.load(f)

#             # Process each segment
#             for segment in transcription.get("segments", []):
#                 rel_start = segment.get("start")
#                 rel_end = segment.get("end")
#                 text = segment.get("text")

#                 if rel_start is None or rel_end is None or not text:
#                     print(f"Warning: Incomplete segment in {adjusted_file}: {segment}")
#                     continue

#                 # Calculate absolute times
#                 abs_start = recording_start + timedelta(seconds=rel_start)
#                 abs_end = recording_start + timedelta(seconds=rel_end)

#                 # Create segment dictionary
#                 segment_dict = {
#                     "username": username,
#                     "start": abs_start,  # datetime object
#                     "end": abs_end,      # datetime object
#                     "text": text
#                 }
#                 all_segments.append(segment_dict)

#     # Sort segments using the new sort_segments function
#     return sort_segments(all_segments, sort_key)


# # Example usage
# if __name__ == "__main__":
#     # Default sorting by start time
#     # unified_object = create_unified_transcription_object()  ## Optional: Sort by end time
#     unified_object = create_unified_transcription_object(sort_key='end')

#     unified_obj_path = '/content/tiny-tutorials/discord-transcription-bot/recordings/unified_object.json'
#     final_transcript_path = '/content/tiny-tutorials/discord-transcription-bot/recordings/transcript.txt'

#     # Save JSON with datetime converted to string
#     with open(unified_obj_path, 'w') as f:
#         json.dump(unified_object, f, indent=2, default=custom_serializer)

In [ ]:
#@title transform_to_transcript
from datetime import datetime

def transform_to_transcript(data):
    # List to store groups of consecutive entries
    groups = []
    current_group = None

    # Step 1: Group consecutive entries by speaker
    for entry in data:
        if current_group is None or entry['username'] == current_group['speaker']:
            # If no current group exists, start a new one
            if current_group is None:
                current_group = {
                    'speaker': entry['username'],
                    'start_time': entry['start'],
                    'texts': [entry['text'].strip()]
                }
            # If speaker is the same, append text to current group
            else:
                current_group['texts'].append(entry['text'].strip())
        else:
            # Speaker changed, save current group and start a new one
            groups.append(current_group)
            current_group = {
                'speaker': entry['username'],
                'start_time': entry['start'],
                'texts': [entry['text'].strip()]
            }

    # Don’t forget to append the last group
    if current_group is not None:
        groups.append(current_group)

    # Step 2: Format each group into transcript lines
    transcript_lines = []
    for i, group in enumerate(groups, 1):  # Start numbering at 1
        # Convert the group's start time to HH:MM:SS format
        start_time = datetime.fromisoformat(group['start_time']).strftime("%H:%M:%S")
        # Format the entry: number, [user @ time], and all texts joined with spaces
        entry_text = f"{i} [{group['speaker']} @ {start_time}]\n{' '.join(group['texts'])}\n\n"
        transcript_lines.append(entry_text)

    # Step 3: Join all lines with newlines
    return "\n".join(transcript_lines)

# Example usage with the rest of your code
# Load JSON back into a Python object
with open(unified_obj_path, 'r') as f:
    unified_object = json.load(f)

# Generate transcript
transcript = transform_to_transcript(unified_object)

# Save transcript to file
with open(final_transcript_path, 'w') as f:
    f.write(transcript)

# Print for verification
print(f"{MAGENTA}\nTransformed transcript saved as 'transcript.txt':{RESET}")
print(transcript)


Transformed transcript saved as 'transcript.txt':
1 [runfishrun @ 13:25:44]
Good.


2 [lu_xo @ 13:25:44]
Hallo, ich begrüße dich. Wir wollen hier kurz eine 5 Minuten über sich machen, wie wir morgen vorgehen an unserem Gespräch mit der PPL von Alberto von Fein von Anschüsse Alberto PPL. In dieser kurzen Übersicht bedarf werden wir die Zeit raster anschauen. Das heißt, sehr fünf Minuten haben wir, wie ein warmes Willkommen, nicht direkt auf Business eingehen, erst mal eine kurze Vorstellung. Wer ich bin, dann hatte ich noch Fragen zu dir, vielleicht kafforferieren, direkt in warm welcome rein, als zweitens nehmen, zeigen wir uns auf Fragebung, denn um uns zu präsentieren, wir sitzen mit dem Kunden einem möglichst zu deinem Konfort Raum der Ruhe, einem Raum der ergehen hat seine Business macht, fragen wir in die 20 Minuten diese Fragen, Themenblocker, lassen hier Zusatzwünsche offene genug Zeitraum für den Kunden zu studieren, dass es sich auch die Wünsche noch notieren kann, welche wir

In [ ]:
# @title ___transform_to_transcript (V.debug )

import json
from datetime import datetime

# Custom function to handle datetime serialization
def custom_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to an ISO 8601 string
    raise TypeError(f"Type {type(obj)} not serializable")

# Save JSON with datetime converted to string
with open('unified_object.json', 'w') as f:
    json.dump(unified_object, f, indent=2, default=custom_serializer)


import json

# Load JSON back into a Python object
with open('unified_object.json', 'r') as f:
    unified_object = json.load(f)




# Transform to simplified transcript format
def transform_to_transcript(data):
    transcript_lines = []
    for i, entry in enumerate(data, 1):
        # Convert start time to hh:mm:ss format
        start_time = datetime.fromisoformat(entry["start"]).strftime("%H:%M:%S")
        # Create entry with username and start time in header
        entry_text = f"{i}\n"
        entry_text += f"[{entry['username']} @ {start_time}]\n"
        entry_text += f"{entry['text'].strip()}\n"
        transcript_lines.append(entry_text)
    return "\n".join(transcript_lines)

# Generate transcript
transcript = transform_to_transcript(unified_object)

# Save transcript to file
with open('transcript.txt', 'w') as f:
    f.write(transcript)

# Print both formats for verification
print("Original JSON saved as 'unified_object.json':")
# print(json.dumps(unified_object, indent=2))
print("\nTransformed transcript saved as 'transcript.txt':")
print(transcript)

Original JSON saved as 'unified_object.json':

Transformed transcript saved as 'transcript.txt':
1
[runfishrun @ 13:25:44]
Good.

2
[lu_xo @ 13:25:44]
Hallo, ich begrüße dich.

3
[lu_xo @ 13:25:48]
Wir wollen hier kurz eine 5 Minuten über sich machen, wie wir morgen vorgehen an unserem Gespräch mit der PPL von Alberto von Fein von Anschüsse Alberto PPL.

4
[lu_xo @ 13:26:00]
In dieser kurzen Übersicht bedarf werden wir die Zeit raster anschauen.

5
[lu_xo @ 13:26:05]
Das heißt, sehr fünf Minuten haben wir, wie ein warmes Willkommen, nicht direkt auf Business eingehen, erst mal eine kurze Vorstellung.

6
[lu_xo @ 13:26:14]
Wer ich bin, dann hatte ich noch Fragen zu dir, vielleicht kafforferieren, direkt

7
[lu_xo @ 13:26:20]
in warm welcome rein, als zweitens nehmen, zeigen wir uns auf Fragebung, denn um uns zu präsentieren, wir sitzen mit dem Kunden einem möglichst zu deinem Konfort Raum der Ruhe, einem Raum der ergehen hat seine Business macht, fragen wir in die 20 Minuten diese Frage

In [ ]:
'''
For downloaed we zip it.
(Google Colab doesnt support download and upload of directories.)
'''

!zip -r recordings.zip recordings

updating: recordings/ (stored 0%)
updating: recordings/unified_object.json (deflated 81%)
updating: recordings/runfishrun/ (stored 0%)
updating: recordings/runfishrun/metadata.json (deflated 61%)
updating: recordings/transcript.txt (deflated 64%)
updating: recordings/lu_xo/ (stored 0%)
updating: recordings/lu_xo/metadata.json (deflated 61%)
updating: recordings/runfishrun/session_20250325_13M53.wav (deflated 90%)
updating: recordings/runfishrun/session_20250325_13M53_transcription_adjusted.txt (deflated 90%)
updating: recordings/runfishrun/session_20250325_13M53_transcription.txt (deflated 90%)
updating: recordings/lu_xo/session_20250325_13M53.wav (deflated 61%)
updating: recordings/lu_xo/session_20250325_13M53_transcription_adjusted.txt (deflated 90%)
updating: recordings/lu_xo/session_20250325_13M53_transcription.txt (deflated 89%)


In [ ]:
# !unzip recordings.zip -d recordings_unzip


### GROQ

In [ ]:
%%capture
!pip install groq

In [ ]:
GROQ_KEY = env.get("GROQ_API_KEY_1")

In [ ]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": ""
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


##<font color='gold'>Further Information

- comparison of some models (e.g. whisper, whisperX, etc.) https://github.com/SYSTRAN/faster-whisper

- Another Notebook with utilizing: [WhisperX.ipynb](https://colab.research.google.com/github/cnbeining/Whisper_Notebook/blob/master/WhisperX.ipynb#scrollTo=VuO7n7Xa7myj)
    
    - GPU setting
    - WhisperX
    - single audio file
    - translation to 31 languages, fileformat: SRT (SubRip SubTitle)
```
1
00:00:05,000 --> 00:00:10,000
Hello, welcome to our video!
```
```
2
00:00:12,000 --> 00:00:15,000
Today, we will talk about translation.
```

- Another Notebook with utilizing: [WhisperX_Speaker_Diarization.ipynb](https://colab.research.google.com/drive/1IHum-j2AOjVOs_ZoqJ5yBUjf1kI4SLmt?usp=sharing)
    - [Youtube-Video: *Multi Speaker Transcription with Speaker IDs with Local Whisper*](https://www.youtube.com/watch?v=SAIsk0i7KgU)
    - `load_align_model` to get word-level timestamps (like I did)
    - Diarization (multiple persons in one audio file)
        - `diarize_model(audio, min_speakers=2, max_speakers=2)` #or auto-detect


### Diarization
- [pyannote.audio speaker diarization toolkit](https://github.com/pyannote/pyannote-audio)

### real-time streaming automatic speech recognition (ASR).
- https://www.reddit.com/r/Python/comments/170iwzc/i_developed_a_realtime_speech_to_text_library/
    - https://www.youtube.com/watch?v=BKOMqKp436Q
    - https://github.com/KoljaB/RealtimeSTT
- https://www.youtube.com/watch?v=_spinzpEeFM
- https://www.youtube.com/watch?v=k6nIxWGdrS4!
- https://www.youtube.com/watch?v=VpB6bxh4deM


##<font color='gold'>whisperx

In [ ]:
# !pip install pydub
!pip install whisperx

In [ ]:
!pwd

/content/tiny-tutorials/discord-transcription-bot


Quite different implementation:
- https://colab.research.google.com/github/cnbeining/Whisper_Notebook/blob/master/WhisperX.ipynb#scrollTo=1JJ0AvEd2lrd

In [ ]:
# Whisper is on large-v2 by default
model_size = 'tiny' #@param ["tiny", "small", "medium", "large-v2", "tiny.en", "small.en", "medium.en"]


In [ ]:
import whisperx
import gc
audio_file_language = 'en'
device = "cpu"
audio_file = "audio.mp3"
audio_file = 'eng_2.m4a'
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model(model_size, device, compute_type='auto')#compute_type)


No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


In [ ]:

# save model to local path (optional)
# model_dir = "/path/"
# model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size, language='en')
print(f'THIS IS ::::::      {result["language"]}')
if result["language"]:
  input_language = result["language"]
else:
  input_language = 'en'#audio_file_language
print(result["segments"]) # before alignment

# # delete model if low on GPU resources
# # import gc; gc.collect(); torch.cuda.empty_cache(); del model

# # 2. Align whisper output
# model_a, metadata = whisperx.load_align_model(language_code=input_language, device=device)
# result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

# print(result["segments"]) # after alignment

# # delete model if low on GPU resources
# # import gc; gc.collect(); torch.cuda.empty_cache(); del model_a

# # 3. Assign speaker labels
# diarize_model = whisperx.DiarizationPipeline(use_auth_token=YOUR_HF_TOKEN, device=device)

# # add min/max number of speakers if known
# diarize_segments = diarize_model(audio)
# # diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

# result = whisperx.assign_word_speakers(diarize_segments, result)
# print(diarize_segments)
# print(result["segments"]) # segments are now assigned speaker IDs

RuntimeError: Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --ena

In [ ]:
for seg in result["segments"]:
  print(seg['text'])

In [ ]:
# import nest_asyncio
# nest_asyncio.apply()

# import asyncio
# import whisper
import model_config

# Load the model
# model = whisper.load_model("base")
# print("Whisper model loaded successfully!")

# Assign the model to the configuration
# model_config.model = whisper.load_model("base")
model_config.model = model
# device = "cpu"
# model = whisperx.load_model("large-v2", device, compute_type="float32")



Bot is NOT (!) running. 🛑


In [ ]:
# model_config.model = model
print("Model assigned to configuration")

# Execute main.py
exec(open("main.py").read())

# Global stop flag
running = True

async def keep_alive():
    global running
    while running:
        await asyncio.sleep(1)  # Keep the loop alive without blocking

# Start the bot and the keep-alive task
loop = asyncio.get_event_loop()
bot_task = loop.create_task(run_bot())
keep_alive_task = loop.create_task(keep_alive())

get_bot_status()

Model assigned to configuration
This script is meant to be run from the notebook.
Bot is RUNNING. 🔴 


In [ ]:
model_a, metadata = whisperx.load_align_model(language_code=result["language"],
                                              device=device)



In [ ]:
recording_sessions.items()

dict_items([('20250319_172440', {977530032622829568: {'audio_file': 'recordings/20250319_172440_977530032622829568.wav', 'username': 'runfishrun', 'discriminator': '0', 'voice_channel': 'Lernraum 1', 'timestamp': '2025-03-19 17:24:40'}, 1351514015549100104: {'audio_file': 'recordings/20250319_172440_1351514015549100104.wav', 'username': 'test_acc_runfish', 'discriminator': '0', 'voice_channel': 'Lernraum 1', 'timestamp': '2025-03-19 17:24:40'}})])

In [ ]:
# After running the bot and making some recordings
for session_id, session_data in recording_sessions.items():
    print(f"Session {session_id}:")
    for user_id, data in session_data.items():
        print(f"  User {user_id}: {data['username']}#{data['discriminator']}")
        print(f"    Audio File: {data['audio_file']}")

        audio = whisperx.load_audio(data['audio_file'])
        result = model.transcribe(audio, batch_size=batch_size, language='en')
        print(result["segments"]) # before alignment
        # 2. Align whisper output
        result = whisperx.align(result["segments"], model_a,
                                metadata,
                                audio,
                                device,
                                return_char_alignments=False)

        print(result["segments"]) # after alignment
        for res in result["segments"]:
          for word in res['words']:
            print(word['word'])
        print(f"    Voice Channel: {data['voice_channel']}")
        print(f"    Timestamp: {data['timestamp']}")

Session 20250319_173433:
  User 977530032622829568: runfishrun#0
    Audio File: recordings/20250319_173433_977530032622829568.wav
[{'text': " Hi. My name is David. Great that you're trying to chat. Okay. Great. Let's talk.", 'start': 0.031, 'end': 20.804}]
[{'start': 0.051, 'end': 0.431, 'text': ' Hi.', 'words': [{'word': 'Hi.', 'start': 0.051, 'end': 0.431, 'score': np.float64(0.89)}]}, {'start': 1.432, 'end': 2.613, 'text': 'My name is David.', 'words': [{'word': 'My', 'start': 1.432, 'end': 1.592, 'score': np.float64(0.947)}, {'word': 'name', 'start': 1.632, 'end': 1.872, 'score': np.float64(0.841)}, {'word': 'is', 'start': 1.912, 'end': 2.032, 'score': np.float64(0.859)}, {'word': 'David.', 'start': 2.152, 'end': 2.613, 'score': np.float64(0.942)}]}, {'start': 3.053, 'end': 4.554, 'text': "Great that you're trying to chat.", 'words': [{'word': 'Great', 'start': 3.053, 'end': 3.353, 'score': np.float64(0.992)}, {'word': 'that', 'start': 3.393, 'end': 3.533, 'score': np.float64(0.91

In [ ]:
#@title pingon
def create_pingpong_dialogue(recording_sessions, base_timestamp=None):
    """
    Convert recording session data into a chronologically ordered ping-pong dialogue format.

    Args:
        recording_sessions: Dictionary of session data keyed by session_id
        base_timestamp: Optional timestamp to use as reference point for relative timing
                        If None, will use the earliest timestamp in the sessions

    Returns:
        List of dialogue turns in chronological order with absolute and relative timing
    """
    # Step 1: Extract all user utterances with their timing information
    all_utterances = []

    for session_id, session_data in recording_sessions.items():
        session_base_time = None

        # Find the earliest timestamp in this session to use as session reference
        for user_id, data in session_data.items():
            timestamp_str = data['timestamp']
            timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
            timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")

            if session_base_time is None or timestamp < session_base_time:
                session_base_time = timestamp

        # Process each user's data
        for user_id, data in session_data.items():
            # Get user info
            username = f"{data['username']}#{data['discriminator']}"
            display_name = data['username'].split('_')[0]  # Extract name before # if any

            # Get timestamp
            timestamp_str = data['timestamp']
            timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")

            # Process segments
            segments = data.get('aligned_segments', [])

            # If segments aren't available, try to get them from the transcription result
            if not segments and 'transcription_result' in data:
                segments = data['transcription_result'].get('segments', [])

            # Process each segment
            for segment in segments:
                utterance = {
                    'user_id': user_id,
                    'username': username,
                    'display_name': display_name,
                    'text': segment['text'].strip(),
                    'timestamp': timestamp_str,
                    'absolute_start': segment['start'],  # Time from start of this user's audio
                    'absolute_end': segment['end'],      # Time from start of this user's audio
                    'session_id': session_id,
                    'voice_channel': data['voice_channel']
                }

                # Calculate relative time (seconds from session start)
                session_offset = (timestamp - session_base_time).total_seconds()
                utterance['relative_start'] = session_offset + segment['start']
                utterance['relative_end'] = session_offset + segment['end']

                if 'words' in segment:
                    utterance['words'] = segment['words']

                all_utterances.append(utterance)

    # Step 2: Sort all utterances by their relative start time
    all_utterances.sort(key=lambda x: x['relative_start'])

    # Step 3: Merge consecutive utterances from the same user if they're close in time
    merged_utterances = []
    current_utterance = None

    max_gap = 2.0  # Maximum gap in seconds between utterances to consider them part of the same turn

    for utterance in all_utterances:
        if current_utterance is None:
            # First utterance
            current_utterance = utterance.copy()
            continue

        if (utterance['user_id'] == current_utterance['user_id'] and
            utterance['relative_start'] - current_utterance['relative_end'] <= max_gap):
            # Same user and close in time - merge
            current_utterance['text'] += " " + utterance['text']
            current_utterance['absolute_end'] = utterance['absolute_end']
            current_utterance['relative_end'] = utterance['relative_end']

            # Merge words if available
            if 'words' in utterance and 'words' in current_utterance:
                current_utterance['words'].extend(utterance['words'])
        else:
            # Different user or too far apart - start new utterance
            merged_utterances.append(current_utterance)
            current_utterance = utterance.copy()

    # Add the last utterance
    if current_utterance:
        merged_utterances.append(current_utterance)

    # Step 4: Create the final ping-pong format
    pingpong_dialogue = []

    # Find global base timestamp if not provided
    if base_timestamp is None:
        if merged_utterances:
            base_timestamp = min(u['relative_start'] for u in merged_utterances)
        else:
            base_timestamp = 0

    for utterance in merged_utterances:
        # Calculate global relative timing
        global_start = utterance['relative_start'] - base_timestamp
        global_end = utterance['relative_end'] - base_timestamp

        turn = {
            'speaker': {
                'name': utterance['display_name'],
                'user_id': utterance['user_id'],
                'username': utterance['username']
            },
            'text': utterance['text'],
            'timestamp': utterance['timestamp'],
            'channel': utterance['voice_channel'],
            'timing': {
                'start': global_start,
                'end': global_end,
                'duration': global_end - global_start
            },
            'audio_timing': {
                'start': utterance['absolute_start'],
                'end': utterance['absolute_end']
            },
            'session_id': utterance['session_id']
        }

        if 'words' in utterance:
            turn['words'] = utterance['words']
            turn['word_count'] = len(utterance['words'])

        pingpong_dialogue.append(turn)

    return pingpong_dialogue


# Example usage:
# Assuming the data is structured as follows after WhisperX processing:
# """
# recording_sessions = {
#     "20250319_173433": {
#         "977530032622829568": {
#             "username": "runfishrun",
#             "discriminator": "0",
#             "audio_file": "recordings/20250319_173433_977530032622829568.wav",
#             "voice_channel": "Lernraum 1",
#             "timestamp": "2025-03-19 17:34:33",
#             "aligned_segments": [
#                 {
#                     "start": 0.051,
#                     "end": 0.431,
#                     "text": " Hi.",
#                     "words": [{"word": "Hi.", "start": 0.051, "end": 0.431, "score": 0.89}]
#                 },
#                 # ... other segments
#             ]
#         },
#         "1351514015549100104": {
#             "username": "test_acc_runfish",
#             "discriminator": "0",
#             "audio_file": "recordings/20250319_173433_1351514015549100104.wav",
#             "voice_channel": "Lernraum 1",
#             "timestamp": "2025-03-19 17:34:33",
#             "aligned_segments": [
#                 # ... segments
#             ]
#         }
#     }
# }

dialogue = create_pingpong_dialogue(recording_sessions)
print(dialogue)

AttributeError: module 'datetime' has no attribute 'strptime'

In [ ]:
import datetime

In [ ]:
#@title Ping-pong dialogue structured as a list of dictionaries
# Each dictionary represents a conversational turn

dialogue = [
    {
        "user_id": "977530032622829568",
        "username": "runfishrun#0",
        "timestamp": "2025-03-19 17:34:33",
        "voice_channel": "Lernraum 1",
        "audio_file": "recordings/20250319_173433_977530032622829568.wav",
        "text": "Hi. My name is David. Great that you're trying to chat. Okay. Great. Let's talk.",
        "words": [
            {"word": "Hi.", "start": 0.051, "end": 0.431, "score": 0.89},
            {"word": "My", "start": 1.432, "end": 1.592, "score": 0.947},
            {"word": "name", "start": 1.632, "end": 1.872, "score": 0.841},
            {"word": "is", "start": 1.912, "end": 2.032, "score": 0.859},
            {"word": "David.", "start": 2.152, "end": 2.613, "score": 0.942},
            {"word": "Great", "start": 3.053, "end": 3.353, "score": 0.992},
            {"word": "that", "start": 3.393, "end": 3.533, "score": 0.917},
            {"word": "you're", "start": 3.553, "end": 3.713, "score": 0.586},
            {"word": "trying", "start": 3.753, "end": 4.034, "score": 0.279},
            {"word": "to", "start": 4.054, "end": 4.154, "score": 0.464},
            {"word": "chat.", "start": 4.194, "end": 4.554, "score": 0.828},
            {"word": "Okay.", "start": 17.982, "end": 18.543, "score": 0.823},
            {"word": "Great.", "start": 18.643, "end": 19.103, "score": 0.894},
            {"word": "Let's", "start": 20.024, "end": 20.284, "score": 0.838},
            {"word": "talk.", "start": 20.384, "end": 20.624, "score": 0.996}
        ],
        "segments": [
            {
                "start": 0.051,
                "end": 0.431,
                "text": " Hi.",
                "words": [{"word": "Hi.", "start": 0.051, "end": 0.431, "score": 0.89}]
            },
            {
                "start": 1.432,
                "end": 2.613,
                "text": "My name is David.",
                "words": [
                    {"word": "My", "start": 1.432, "end": 1.592, "score": 0.947},
                    {"word": "name", "start": 1.632, "end": 1.872, "score": 0.841},
                    {"word": "is", "start": 1.912, "end": 2.032, "score": 0.859},
                    {"word": "David.", "start": 2.152, "end": 2.613, "score": 0.942}
                ]
            },
            {
                "start": 3.053,
                "end": 4.554,
                "text": "Great that you're trying to chat.",
                "words": [
                    {"word": "Great", "start": 3.053, "end": 3.353, "score": 0.992},
                    {"word": "that", "start": 3.393, "end": 3.533, "score": 0.917},
                    {"word": "you're", "start": 3.553, "end": 3.713, "score": 0.586},
                    {"word": "trying", "start": 3.753, "end": 4.034, "score": 0.279},
                    {"word": "to", "start": 4.054, "end": 4.154, "score": 0.464},
                    {"word": "chat.", "start": 4.194, "end": 4.554, "score": 0.828}
                ]
            },
            {
                "start": 17.982,
                "end": 18.543,
                "text": "Okay.",
                "words": [{"word": "Okay.", "start": 17.982, "end": 18.543, "score": 0.823}]
            },
            {
                "start": 18.643,
                "end": 19.103,
                "text": "Great.",
                "words": [{"word": "Great.", "start": 18.643, "end": 19.103, "score": 0.894}]
            },
            {
                "start": 20.024,
                "end": 20.624,
                "text": "Let's talk.",
                "words": [
                    {"word": "Let's", "start": 20.024, "end": 20.284, "score": 0.838},
                    {"word": "talk.", "start": 20.384, "end": 20.624, "score": 0.996}
                ]
            }
        ]
    },
    {
        "user_id": "1351514015549100104",
        "username": "test_acc_runfish#0",
        "timestamp": "2025-03-19 17:34:33",
        "voice_channel": "Lernraum 1",
        "audio_file": "recordings/20250319_173433_1351514015549100104.wav",
        "text": "I, my name is August. Hi.",
        "words": [
            {"word": "I,", "start": 0.403, "end": 0.423, "score": 0.002},
            {"word": "my", "start": 0.524, "end": 0.704, "score": 0.863},
            {"word": "name", "start": 0.765, "end": 0.965, "score": 0.998},
            {"word": "is", "start": 1.026, "end": 1.126, "score": 0.755},
            {"word": "August.", "start": 1.407, "end": 1.989, "score": 0.852},
            {"word": "Hi.", "start": 3.415, "end": 3.535, "score": 0.71}
        ],
        "segments": [
            {
                "start": 0.403,
                "end": 1.989,
                "text": " I, my name is August.",
                "words": [
                    {"word": "I,", "start": 0.403, "end": 0.423, "score": 0.002},
                    {"word": "my", "start": 0.524, "end": 0.704, "score": 0.863},
                    {"word": "name", "start": 0.765, "end": 0.965, "score": 0.998},
                    {"word": "is", "start": 1.026, "end": 1.126, "score": 0.755},
                    {"word": "August.", "start": 1.407, "end": 1.989, "score": 0.852}
                ]
            },
            {
                "start": 3.415,
                "end": 3.535,
                "text": "Hi.",
                "words": [{"word": "Hi.", "start": 3.415, "end": 3.535, "score": 0.71}]
            }
        ]
    }
]

# Function to get ping-pong discussion in simplified format
def get_ping_pong_format():
    return [
        {
            "speaker": "David (runfishrun#0)",
            "text": "Hi. My name is David. Great that you're trying to chat. Okay. Great. Let's talk.",
            "timestamp": "2025-03-19 17:34:33"
        },
        {
            "speaker": "August (test_acc_runfish#0)",
            "text": "I, my name is August. Hi.",
            "timestamp": "2025-03-19 17:34:33"
        }
    ]

# Function to get more detailed ping-pong discussion with timing information
def get_detailed_ping_pong_format():
    return [
        {
            "speaker": {
                "name": "David",
                "user_id": "977530032622829568",
                "username": "runfishrun#0"
            },
            "text": "Hi. My name is David. Great that you're trying to chat. Okay. Great. Let's talk.",
            "timestamp": "2025-03-19 17:34:33",
            "channel": "Lernraum 1",
            "word_count": 15,
            "duration": 20.624 - 0.051,  # End time of last word minus start time of first word
            "audio_file": "recordings/20250319_173433_977530032622829568.wav"
        },
        {
            "speaker": {
                "name": "August",
                "user_id": "1351514015549100104",
                "username": "test_acc_runfish#0"
            },
            "text": "I, my name is August. Hi.",
            "timestamp": "2025-03-19 17:34:33",
            "channel": "Lernraum 1",
            "word_count": 6,
            "duration": 3.535 - 0.403,  # End time of last word minus start time of first word
            "audio_file": "recordings/20250319_173433_1351514015549100104.wav"
        }
    ]

In [ ]:
from collections import defaultdict

def process_transcription(recording_sessions):
    user_segments = []

    # Extract words along with user IDs
    for session_id, session_data in recording_sessions.items():
        for user_id, data in session_data.items():
            audio_file = data['audio_file']
            result = model.transcribe(whisperx.load_audio(audio_file), batch_size=batch_size, language='en')
            result = whisperx.align(result["segments"], model_a, metadata, whisperx.load_audio(audio_file), device, return_char_alignments=False)

            for segment in result["segments"]:
                for word in segment['words']:
                    user_segments.append((user_id, word['word']))

    # Order words based on their start and end letters
    user_segments.sort(key=lambda x: (x[1][0], x[1][-1]))

    # Bin words together as long as the user doesn’t switch
    binned_words = defaultdict(list)
    prev_user = None
    temp_list = []

    for user_id, word in user_segments:
        if user_id != prev_user and prev_user is not None:
            binned_words[prev_user].append(temp_list)
            temp_list = []
        temp_list.append(word)
        prev_user = user_id

    if temp_list:
        binned_words[prev_user].append(temp_list)

    # Generate report
    report = "Transcription Report\n\n"
    for user_id, bins in binned_words.items():
        report += f"User {user_id}:\n"
        for bin_words in bins:
            report += "  " + " ".join(bin_words) + "\n"
        report += "\n"

    return report

# Example usage:
report = process_transcription(recording_sessions)
print(report)

Transcription Report

User 1351514015549100104:
  August.
  Hi. I,
  is my
  name

User 977530032622829568:
  David. Great. Great Hi.
  Let's My Okay. chat. is
  name
  talk. trying to that you're




## <font color='gold'>try sth. new

In [ ]:
!pip install opuslib webrtcvad scipy numpy whisper openai-whisper python-dotenv

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title requirements.txt
%%capture

content = """py-cord
python-dotenv
openai-whisper""" # takes quite long...

with open("requirements.txt", "w") as file:
    file.write(content)

print("requirements.txt has been created.")

!pip install -r ./requirements.txt
!pip install PyNaCl
!pip install nest_asyncio
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import discord
import opuslib
import webrtcvad
import scipy.signal
import numpy as np
import whisper
import asyncio
from collections import deque
from dotenv import load_dotenv
from os import environ as env

# Initialize the bot
bot = discord.Bot()
# bot = discord.Bot()

connections = {}  # Track voice clients per guild
load_dotenv()

# Load the Whisper model globally (using "base" for a balance of speed and accuracy)
model = whisper.load_model("base")

class RealTimeTranscriptionSink(discord.sinks.Sink):
    def __init__(self, channel):
        """Initialize the custom sink with a channel to send transcriptions."""
        self.channel = channel
        self.vad = webrtcvad.Vad(3)  # Aggressive VAD mode
        self.silence_threshold = 50  # 1 second of silence (50 frames * 20ms)
        self.users = {}  # Store state per user: {user_id: {"decoder", "buffer", "silence_count", "recording"}}

    def write(self, data, user):
        """Process each audio packet in real-time."""
        user_id = user.id
        # Initialize user state if not present
        if user_id not in self.users:
            self.users[user_id] = {
                "decoder": opuslib.Decoder(48000, 2),  # 48kHz, stereo
                "buffer": deque(),  # Buffer for audio frames
                "silence_count": 0,  # Count consecutive silence frames
                "recording": False  # Track recording state
            }

        user_state = self.users[user_id]
        decoder = user_state["decoder"]
        # Decode Opus packet to PCM (16-bit, 48kHz, stereo)
        pcm = decoder.decode(data.data, frame_size=960)  # 20ms frame
        # Downmix stereo to mono for VAD
        pcm_mono = np.frombuffer(pcm, dtype=np.int16).reshape(-1, 2).mean(axis=1).astype(np.int16)
        is_speech = self.vad.is_speech(pcm_mono.tobytes(), 48000)

        # State machine for recording
        if is_speech and not user_state["recording"]:
            # Start recording when speech is detected
            user_state["recording"] = True
            user_state["buffer"].clear()
            user_state["silence_count"] = 0
        elif user_state["recording"]:
            # Append audio frame to buffer
            user_state["buffer"].append(pcm)
            if not is_speech:
                # Increment silence counter during silence
                user_state["silence_count"] += 1
                if user_state["silence_count"] >= self.silence_threshold:
                    # Process buffer and reset state after 1 second of silence
                    self.process_buffer(user_id)
                    user_state["recording"] = False
                    user_state["buffer"].clear()
                    user_state["silence_count"] = 0
            else:
                # Reset silence counter if speech resumes
                user_state["silence_count"] = 0

    def process_buffer(self, user_id):
        """Process the buffered audio and transcribe it."""
        user_state = self.users[user_id]
        buffer = user_state["buffer"]
        if buffer:
            # Combine buffered PCM frames
            pcm_stereo_48khz = b''.join(buffer)
            # Prepare audio for Whisper
            audio = self.prepare_audio_for_whisper(pcm_stereo_48khz)
            # Transcribe the audio
            result = model.transcribe(audio, fp16=False, language='en', sampling_rate=16000)
            text = result['text'].strip()
            if text:
                # Send transcription to the channel asynchronously
                asyncio.create_task(self.channel.send(f"Speaker {user_id}: {text}"))

    def prepare_audio_for_whisper(self, pcm_stereo_48khz):
        """Convert PCM audio to the format expected by Whisper (16kHz, mono, float32)."""
        # Convert stereo PCM to numpy array
        stereo = np.frombuffer(pcm_stereo_48khz, dtype=np.int16).reshape(-1, 2)
        # Downmix to mono by averaging channels
        mono = stereo.mean(axis=1).astype(np.float32) / 32768.0  # Normalize to [-1, 1]
        # Resample from 48kHz to 16kHz
        num_samples = int(len(mono) * 16000 / 48000)
        mono_16khz = scipy.signal.resample(mono, num_samples)
        return mono_16khz

# Bot commands
@bot.command()
async def record(ctx):
    """Start recording and transcribing the voice channel in real-time."""
    voice = ctx.author.voice
    if not voice:
        await ctx.respond("⚠️ You aren't in a voice channel!")
        return
    vc = await voice.channel.connect()
    connections[ctx.guild.id] = vc
    sink = RealTimeTranscriptionSink(ctx.channel)
    vc.start_recording(sink, once_done, ctx.channel)
    await ctx.respond("🔴 Listening to this conversation.")

async def once_done(sink, channel, *args):
    """Callback executed when recording stops."""
    await channel.send("🛑 Recording stopped.")


# @bot.command()
# async def record(ctx):
#     """Start recording and transcribing the voice channel in real-time."""
#     # Tell Discord we're working on it to avoid the timeout
#     await ctx.defer()

#     # Check if the user is in a voice channel
#     voice = ctx.author.voice
#     if not voice:
#         await ctx.followup.send("⚠️ You aren't in a voice channel!")
#         return

#     # Connect to the voice channel
#     vc = await voice.channel.connect()

#     # Set up and start recording (adjust this based on your actual recording setup)
#     connections[ctx.guild.id] = vc
#     sink = RealTimeTranscriptionSink(ctx.channel)
#     vc.start_recording(sink, once_done, ctx.channel)

#     # Send the confirmation message
#     await ctx.followup.send("🔴 Listening to this conversation.")


@bot.command()
async def stop_recording(ctx):
    """Stop recording and disconnect from the voice channel."""
    if ctx.guild.id in connections:
        vc = connections[ctx.guild.id]
        vc.stop_recording()  # Triggers once_done
        del connections[ctx.guild.id]
        await vc.disconnect()
        await ctx.message.delete()  # Clean up the command message
    else:
        await ctx.respond("🚫 Not recording in this guild.")

# Run the bot
bot.run(env.get("DISCORD_BOT_TOKEN"))

ModuleNotFoundError: No module named 'discord'

## Real time ASR

In [ ]:
!pip install faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
from faster_whisper import WhisperModel

In [ ]:
# # Install system dependencies first
# !apt-get update && apt-get install -y portaudio19-dev python3-pyaudio

# # Then install PyAudio
# !pip install pyaudio

In [ ]:
# !pip install pyaudio
import pyaudio

Could not import the PyAudio C module '_portaudio'.


ModuleNotFoundError: No module named '_portaudio'

In [ ]:
# !pip install librosa soundfile

In [ ]:
!pip install ffmpeg-python

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

# Updated JavaScript code for recording
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");
my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = null;  // Changed to null for clarity
var recorder, gumStream;
var recordButton = my_btn;
var recording = false;

var handleSuccess = function(stream) {
  gumStream = stream;
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    };
  };
  recorder.onstop = function() {
    gumStream.getAudioTracks()[0].stop();
  };
  recorder.start();
  recordButton.textContent = "Recording... Press to stop";
  recording = true;
};

recordButton.onclick = function() {
  if (!recording) {
    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
  } else {
    recorder.stop();
    recordButton.textContent = "Press to start recording";
    recording = false;
  }
};

var base64data = null;
window.dataPromise = () => new Promise(resolve => {
  var checkData = setInterval(() => {
    if (base64data !== null) {
      var data = base64data;
      console.log("Promise resolved with data");
      resolve(data);
      base64data = null;
    }
  }, 200);
});
console.log("Script loaded, dataPromise defined");
// Rest of your recording logic...
</script>
"""

# Display the recording button
display(HTML(AUDIO_HTML))

# Function to retrieve the recorded audio
def get_audio():
  audio_data = eval_js("dataPromise")
  binary = b64decode(audio_data.split(',')[1])
  process = (ffmpeg
             .input('pipe:0')
             .output('pipe:1', format='wav')
             .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True))
  output, _ = process.communicate(input=binary)
  return output

# Instructions and audio retrieval
print("Press the button to start recording. Press again to stop.")
print("Wait a few seconds after stopping for the audio to process.")
audio_binary = get_audio()

# Convert to playable audio
audio = Audio(audio_binary, autoplay=False)
display(audio)

Press the button to start recording. Press again to stop.
Wait a few seconds after stopping for the audio to process.


MessageError: DataCloneError: Failed to execute 'postMessage' on 'MessagePort': () => new Promise(resolve => {
  var checkData = setInterval(() => {
    if (base64data !== null) {
      var d...<omitted>...}) could not be cloned.

In [ ]:
def record_chunk(p, stream, file_path, chunk_length=1):
    frames = []
    for _ in range(0, int(16000 / 1024 * chunk_length)):
        data = stream.read(1024)
        frames.append(data)

    wf = wave.open(file_path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(16000)
    wf.writeframes(b''.join(frames))
    wf.close()

def main2():
    # Choose your model settings
    model_size = "medium.en"
    model = WhisperModel(model_size,
                        #  device="cuda",
                         device="cpu",
                        #  compute_type="float16"
                         )

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=1024)

    accumulated_transcription = "" # Initialize an empty string to accumulate transcriptions

    try:
        while True:
            chunk_file = "temp_chunk.wav"
            record_chunk(p, stream, chunk_file)
            transcription = transcribe_chunk(model, chunk_file)
            print(GREEN + transcription + RESET_COLOR)
            os.remove(chunk_file)

            # Append the new transcription to the accumulated transcription
            accumulated_transcription += transcription + " "

    except KeyboardInterrupt:
        print("Stopping...")
        # Write the accumulated transcription to the log file
        with open("log.txt", "w") as log_file:
            log_file.write(accumulated_transcription)
    finally:
        print("LOG:" + accumulated_transcription)
        stream.stop_stream()
        stream.close()
        p.terminate()

if __name__ == "__main__":
    main2()

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import write as wav_write
import io
import ffmpeg
import time

# JavaScript code to record audio chunks
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");
my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = null;
var recorder, gumStream;
var recordButton = my_btn;
var recording = false;
var chunkDuration = 2000; // Increased to 2 seconds for stability

var handleSuccess = function(stream) {
  gumStream = stream;
  recorder = new MediaRecorder(stream, { mimeType: 'audio/webm' }); // Explicit MIME type
  recorder.ondataavailable = function(e) {
    console.log("Data available, size: " + e.data.size);
    var reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      console.log("Base64 data ready: " + base64data.substring(0, 50) + "...");
    };
  };
  recorder.onstop = function() {
    gumStream.getAudioTracks()[0].stop();
    console.log("Recorder stopped");
  };
  recorder.start();
  console.log("Recording started");
  recordButton.textContent = "Recording... Press to stop";
  recording = true;

  // Automatically stop and restart every chunkDuration
  setTimeout(() => {
    if (recording) {
      recorder.stop();
      console.log("Auto-stop triggered");
      navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
    }
  }, chunkDuration);
};

recordButton.onclick = function() {
  if (!recording) {
    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
  } else {
    recorder.stop();
    recordButton.textContent = "Press to start recording";
    recording = false;
    console.log("Manual stop");
  }
};

// Promise to resolve with each chunk of audio data
var dataPromise = () => new Promise(resolve => {
  var checkData = setInterval(() => {
    if (base64data !== null) {
      var data = base64data;
      console.log("Promise resolved with data");
      resolve(data);
      base64data = null; // Reset after resolving
    }
  }, 200); // Increased polling interval
});
dataPromise
</script>
"""

# Display the recording interface
display(HTML(AUDIO_HTML))

# Function to get audio chunk from JavaScript
def get_audio_chunk():
  try:
    audio_data = eval_js("dataPromise()")
    if not audio_data or audio_data == "null":
      raise ValueError("No audio data received from JavaScript")

    # Decode base64 audio data
    binary = b64decode(audio_data.split(',')[1])
    if len(binary) == 0:
      raise ValueError("Decoded audio data is empty")

    # Convert WebM to WAV using ffmpeg
    process = (ffmpeg
               .input('pipe:0', format='webm')
               .output('pipe:1', format='wav')
               .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True))
    output, err = process.communicate(input=binary)

    if not output:
      raise ValueError(f"FFmpeg failed to convert audio: {err.decode()}")

    print(f"Audio chunk retrieved, size: {len(output)} bytes")
    return output
  except Exception as e:
    print(f"Error in get_audio_chunk: {e}")
    return None

# Function to save and verify audio chunk
def save_audio_chunk(audio_data, file_path="temp_chunk.wav"):
  try:
    with open(file_path, "wb") as f:
      f.write(audio_data)
    print(f"Saved audio chunk to {file_path}, size: {len(audio_data)} bytes")
    display(Audio(file_path, autoplay=False))
  except Exception as e:
    print(f"Error saving audio chunk: {e}")

# Main function for real-time audio capture
def main_real_time():
  accumulated_transcription = ""

  print("Starting real-time audio capture. Press the button to begin.")
  print("Click again to stop manually, or let it run for continuous chunks.")

  try:
    while True:
      audio_chunk = get_audio_chunk()
      if audio_chunk:
        save_audio_chunk(audio_chunk)
        accumulated_transcription += " [Audio chunk captured] "
      else:
        print("Skipping invalid chunk")
      time.sleep(2)  # Match chunkDuration for stability

  except KeyboardInterrupt:
    print("Stopping...")
    with open("log.txt", "w") as log_file:
      log_file.write(accumulated_transcription)
    print(f"LOG: {accumulated_transcription}")
  except Exception as e:
    print(f"Error: {e}")
    with open("log.txt", "w") as log_file:
      log_file.write(accumulated_transcription)
    print(f"LOG: {accumulated_transcription}")

# Install required dependencies
!pip install ffmpeg-python

# Run the main function
if __name__ == "__main__":
  main_real_time()

Starting real-time audio capture. Press the button to begin.
Click again to stop manually, or let it run for continuous chunks.
Audio chunk retrieved, size: 69198 bytes
Saved audio chunk to temp_chunk.wav, size: 69198 bytes


Audio chunk retrieved, size: 190158 bytes
Saved audio chunk to temp_chunk.wav, size: 190158 bytes


Audio chunk retrieved, size: 190158 bytes
Saved audio chunk to temp_chunk.wav, size: 190158 bytes


Audio chunk retrieved, size: 190158 bytes
Saved audio chunk to temp_chunk.wav, size: 190158 bytes


In [ ]:
!pip install ffmpeg-python
!pip install faster-whisper

In [ ]:
from faster_whisper import WhisperModel

In [ ]:
from google.colab.output import eval_js
from base64 import b64decode
import ffmpeg



# JavaScript code to record audio chunks
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");
my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = null;
var recorder, gumStream;
var recordButton = my_btn;
var recording = false;
var chunkDuration = 2000; // Increased to 2 seconds for stability

var handleSuccess = function(stream) {
  gumStream = stream;
  recorder = new MediaRecorder(stream, { mimeType: 'audio/webm' }); // Explicit MIME type
  recorder.ondataavailable = function(e) {
    console.log("Data available, size: " + e.data.size);
    var reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      console.log("Base64 data ready: " + base64data.substring(0, 50) + "...");
    };
  };
  recorder.onstop = function() {
    gumStream.getAudioTracks()[0].stop();
    console.log("Recorder stopped");
  };
  recorder.start();
  console.log("Recording started");
  recordButton.textContent = "Recording... Press to stop";
  recording = true;

  // Automatically stop and restart every chunkDuration
  setTimeout(() => {
    if (recording) {
      recorder.stop();
      console.log("Auto-stop triggered");
      navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
    }
  }, chunkDuration);
};

recordButton.onclick = function() {
  if (!recording) {
    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
  } else {
    recorder.stop();
    recordButton.textContent = "Press to start recording";
    recording = false;
    console.log("Manual stop");
  }
};

// Promise to resolve with each chunk of audio data
window.dataPromise = () => new Promise(resolve => {
  var checkData = setInterval(() => {
    if (base64data !== null) {
      var data = base64data;
      console.log("Promise resolved with data");
      resolve(data);
      base64data = null; // Reset after resolving
    }
  }, 200);
});
dataPromise
</script>
"""


def record_chunk():
    # Get base64-encoded WebM audio data from JavaScript
    base64_data = eval_js('dataPromise()')

    # Decode base64 data (strip the "data:audio/webm;base64," prefix)
    webm_data = b64decode(base64_data.split(',')[1])

    # Convert WebM to WAV using ffmpeg
    process = (
        ffmpeg
        .input('pipe:', format='webm')
        .output('pipe:', format='wav', acodec='pcm_s16le', ar='16000', ac=1)
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )
    wav_bytes, _ = process.communicate(input=webm_data)

    # Save to a temporary file
    chunk_file = "temp_chunk.wav"
    with open(chunk_file, 'wb') as f:
        f.write(wav_bytes)

    return chunk_file


import os

def main2():
    # Initialize the Whisper model
    model_size = "medium.en"
    model = WhisperModel(model_size,
                        #  device="cuda",
                         device="cpu",
                        #  compute_type="float16"
                         )

    accumulated_transcription = ""

    try:
        while True:
            # Get the next audio chunk
            chunk_file = record_chunk()

            # Transcribe the chunk
            transcription = transcribe_chunk(model, chunk_file)

            # Print the transcription with color
            print(GREEN + transcription + RESET_COLOR)

            # Clean up the temporary file
            os.remove(chunk_file)

            # Accumulate the transcription
            accumulated_transcription += transcription + " "

    except KeyboardInterrupt:
        print("Stopping...")
        # Save the accumulated transcription to a log file
        with open("log.txt", "w") as log_file:
            log_file.write(accumulated_transcription.strip())

    finally:
        print("LOG:" + accumulated_transcription.strip())

In [ ]:
from IPython.display import display, HTML
display(HTML(AUDIO_HTML))

In [ ]:
main2()

LOG:


MessageError: ReferenceError: dataPromise is not defined

## other chapter

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()

In [ ]:
!pip install faster-whisper ffmpeg-python

In [ ]:
from IPython.display import HTML, Audio, display, clear_output
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read, write as wav_write
import io
import ffmpeg
import time
import threading
import queue
import os
from faster_whisper import WhisperModel
import warnings
import wave
from scipy import signal

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Modified HTML/JavaScript for real-time chunked recording
AUDIO_HTML = """
<script>
// Create UI elements
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var status_div = document.createElement("DIV");
status_div.style.marginTop = "10px";
status_div.innerHTML = "Ready to record";

my_btn.textContent = "Start Recording";
my_div.appendChild(my_btn);
my_div.appendChild(status_div);
document.body.appendChild(my_div);

// Variables for recording
var recorder, gumStream;
var recordButton = my_btn;
var isRecording = false;
var audioChunks = [];
var chunkInterval = 3000; // 3 seconds per chunk
var chunkTimer;
var transcriptionActive = false;

// Queue to store chunks
var chunkQueue = [];

// Function to handle successful media stream
function handleSuccess(stream) {
  gumStream = stream;
  var options = {
    mimeType: 'audio/webm;codecs=opus'
  };

  recorder = new MediaRecorder(stream);

  // Event triggered when data is available
  recorder.ondataavailable = function(e) {
    if (e.data.size > 0) {
      audioChunks.push(e.data);
      processChunk();
    }
  };

  // Start recording
  recorder.start();
  isRecording = true;
  recordButton.textContent = "Stop Recording";
  status_div.innerHTML = "Recording...";

  // Set up timer for chunk collection
  startChunkTimer();
}

// Timer function to collect audio in chunks
function startChunkTimer() {
  chunkTimer = setInterval(function() {
    if (recorder && recorder.state === "recording") {
      recorder.requestData(); // Request data without stopping recorder
    }
  }, chunkInterval);
}

// Process an audio chunk
function processChunk() {
  if (audioChunks.length > 0) {
    var latestChunk = audioChunks.pop();
    audioChunks = []; // Clear for next chunks

    // Create blob from the chunk
    var blob = new Blob([latestChunk], { type: 'audio/webm;codecs=opus' });

    // Convert to base64
    var reader = new FileReader();
    reader.onloadend = function() {
      var base64data = reader.result;
      // Add to queue for Python to process
      chunkQueue.push(base64data);
      status_div.innerHTML = `Recording... (${chunkQueue.length} chunks queued)`;
    };
    reader.readAsDataURL(blob);
  }
}

// Toggle recording on button click
recordButton.onclick = function() {
  if (!isRecording) {
    // Start recording
    transcriptionActive = true;
    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
  } else {
    // Stop recording
    clearInterval(chunkTimer);
    if (recorder) {
      recorder.stop();
      gumStream.getAudioTracks().forEach(track => track.stop());
    }
    isRecording = false;
    transcriptionActive = false;
    recordButton.textContent = "Start Recording";
    status_div.innerHTML = "Recording stopped";
  }
};

// Function to check if there are any chunks available
window.hasNextChunk = function() {
  return chunkQueue.length > 0;
};

// Function to get the next chunk
window.getNextChunk = function() {
  if (chunkQueue.length > 0) {
    return chunkQueue.shift();
  }
  return null;
};

// Function to check if transcription is active
window.isTranscriptionActive = function() {
  return transcriptionActive;
};
</script>
"""

# Improved function to convert base64 audio to audio data
def process_audio_chunk(base64data):
    """Process audio chunk using direct ffmpeg pipe to avoid file format issues"""
    try:
        # Decode base64 data
        binary = b64decode(base64data.split(',')[1])

        # Use ffmpeg to convert to WAV
        # Set explicit audio parameters to ensure consistency
        process = (ffmpeg
            .input('pipe:0')
            .output('pipe:1', format='wav', acodec='pcm_s16le', ar='16000', ac='1')
            .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True)
        )
        output, err = process.communicate(input=binary)

        # Process the WAV data directly without using scipy.io.wavfile
        with io.BytesIO(output) as wav_io:
            # Open using wave library which is more robust for this purpose
            with wave.open(wav_io, 'rb') as wav:
                # Get parameters
                frames = wav.readframes(wav.getnframes())
                rate = wav.getframerate()
                channels = wav.getnchannels()
                width = wav.getsampwidth()

                # Convert to numpy array
                if width == 2:  # 16-bit audio
                    dtype = np.int16
                elif width == 4:  # 32-bit audio
                    dtype = np.int32
                else:
                    dtype = np.int8

                audio = np.frombuffer(frames, dtype=dtype)

                # If stereo, convert to mono by averaging channels
                if channels == 2:
                    audio = audio.reshape(-1, 2).mean(axis=1).astype(dtype)

                return audio, rate

    except Exception as e:
        print(f"Error processing audio chunk: {e}")
        # Return a silent audio chunk rather than None
        return np.zeros(16000, dtype=np.int16), 16000

# Alternate method to process audio if traditional method fails
def process_audio_chunk_alternate(base64data):
    """Alternative method to process audio chunks if the primary method fails"""
    try:
        # Decode base64 data
        binary = b64decode(base64data.split(',')[1])

        # Save to a temporary file
        with open('/tmp/temp_audio.webm', 'wb') as f:
            f.write(binary)

        # Use ffmpeg to convert to WAV file
        os.system('ffmpeg -i /tmp/temp_audio.webm -ar 16000 -ac 1 -c:a pcm_s16le /tmp/temp_audio.wav -y -loglevel error')

        # Read the WAV file
        with wave.open('/tmp/temp_audio.wav', 'rb') as wav:
            frames = wav.readframes(wav.getnframes())
            rate = wav.getframerate()
            audio = np.frombuffer(frames, dtype=np.int16)

        # Clean up
        os.remove('/tmp/temp_audio.webm')
        os.remove('/tmp/temp_audio.wav')

        return audio, rate

    except Exception as e:
        print(f"Error in alternate audio processing: {e}")
        return np.zeros(16000, dtype=np.int16), 16000

# Function to initialize the transcription model
def initialize_model(model_size="base"):
    print(f"Loading {model_size} Whisper model...")
    # Use either CPU or CUDA depending on availability
    model = WhisperModel(model_size, device="cuda" if "CUDA_VISIBLE_DEVICES" in os.environ else "cpu")
    print("Model loaded!")
    return model

# Chunked recording and real-time transcription
def record_chunk():
    """Get a single chunk of audio from the JavaScript interface"""
    # Check if a chunk is available
    has_chunk = eval_js("hasNextChunk()")

    if has_chunk:
        # Get the next audio chunk
        chunk_data = eval_js("getNextChunk()")
        if chunk_data:
            try:
                # Try primary method first
                audio, sr = process_audio_chunk(chunk_data)
                if audio is None or len(audio) == 0:
                    raise Exception("Primary method failed")
                return audio, sr
            except Exception as e:
                print(f"Trying alternate method: {e}")
                # Try alternate method if primary fails
                return process_audio_chunk_alternate(chunk_data)

    return None, None

# Create a function to update transcription display
def update_transcription_display(text):
    from IPython.display import display as ipython_display
    from IPython.display import HTML as ipython_HTML
    ipython_display(ipython_HTML(f"<script>document.getElementById('transcription-output').innerHTML = '<h3>Transcription:</h3>{text}';</script>"))

# Function to save audio data to a temporary file for transcription
def save_audio_for_transcription(audio, sr):
    """Save audio data to a temporary file in the correct format for transcription"""
    temp_file = io.BytesIO()

    # Ensure audio is in the correct format
    if audio.dtype != np.int16:
        audio = audio.astype(np.int16)

    # Resample if necessary (whisper expects 16kHz)
    if sr != 16000:
        audio = signal.resample(audio, int(len(audio) * 16000 / sr))
        sr = 16000

    try:
        # Create WAV file
        with wave.open(temp_file, 'wb') as wav:
            wav.setnchannels(1)  # mono
            wav.setsampwidth(2)  # 16-bit
            wav.setframerate(sr)
            wav.writeframes(audio.tobytes())

        temp_file.seek(0)
        return temp_file
    except Exception as e:
        print(f"Error saving audio: {e}")
        return None

# Main function for real-time transcription
def realtime_transcription(model_size="base", chunk_processing_queue=None):
    """
    Perform real-time transcription using Whisper model

    Args:
        model_size: Size of the Whisper model to use
        chunk_processing_queue: Queue to store processed chunks (optional)
    """
    # Display the recording interface
    display(HTML(AUDIO_HTML))

    print("Initializing transcription model...")
    model = initialize_model(model_size)

    # Create output area for transcriptions
    display(HTML("<div id='transcription-output' style='margin-top: 20px; padding: 10px; border: 1px solid #ccc; min-height: 100px;'><h3>Transcription:</h3></div>"))

    print("Ready! Click the button to start/stop recording.")
    print("Transcription will appear below as you speak.")

    # Initialize variables for transcription
    transcription_text = ""
    segment_history = []

    # Main loop
    try:
        while True:
            # Check if transcription is active
            is_active = eval_js("isTranscriptionActive()")

            if is_active:
                # Get a chunk of audio
                audio, sr = record_chunk()

                if audio is not None and len(audio) > 0:
                    # Ensure the audio has enough content to transcribe
                    if len(audio) < 1000:  # Very short audio, likely empty
                        continue

                    # Save audio to temporary file
                    temp_file = save_audio_for_transcription(audio, sr)
                    if temp_file is None:
                        continue

                    try:
                        # Transcribe the chunk
                        segments, info = model.transcribe(temp_file, beam_size=5, language="en")

                        # Process segments
                        has_content = False
                        for segment in segments:
                            segment_text = segment.text.strip()
                            if segment_text:
                                has_content = True
                                segment_history.append(segment_text)
                                # Keep only the last 10 segments to avoid too much text
                                if len(segment_history) > 10:
                                    segment_history.pop(0)

                                # Update the transcription display
                                full_text = " ".join(segment_history)
                                update_transcription_display(full_text)

                                # Add to queue if provided
                                if chunk_processing_queue is not None:
                                    chunk_processing_queue.put(segment_text)

                        if not has_content:
                            print("No transcription content in this chunk")

                    except Exception as e:
                        print(f"Error during transcription: {e}")

            # Small delay to avoid hammering the browser
            time.sleep(0.1)

    except KeyboardInterrupt:
        print("Transcription stopped by user.")

    return segment_history

# Sample usage function for the entire system
def main():
    # Create a queue for processing transcribed chunks
    chunk_queue = queue.Queue()

    # Create a thread for any additional processing of transcribed text
    def process_transcription():
        while True:
            try:
                text = chunk_queue.get(timeout=1)
                # Do something with the transcribed text
                # For example, analyze sentiment, extract commands, etc.
                print(f"Processing: {text}")
                chunk_queue.task_done()
            except queue.Empty:
                time.sleep(0.1)
            except Exception as e:
                print(f"Error in processing thread: {e}")

    # Start processing in a separate thread
    processing_thread = threading.Thread(target=process_transcription)
    processing_thread.daemon = True
    processing_thread.start()

    # Start the real-time transcription
    try:
        return realtime_transcription(model_size="base", chunk_processing_queue=chunk_queue)
    except Exception as e:
        print(f"Error in transcription: {e}")
        return None

In [ ]:
transcription = main()

Initializing transcription model...
Loading base Whisper model...
Model loaded!


Ready! Click the button to start/stop recording.
Transcription will appear below as you speak.
No transcription content in this chunk
Error processing audio chunk: 
No transcription content in this chunk
Error processing audio chunk: 
No transcription content in this chunk
Transcription stopped by user.


In [ ]:
from IPython.display import HTML, Audio, display, clear_output
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read, write as wav_write
import io
import ffmpeg
import time
import threading
import queue
import os
from faster_whisper import WhisperModel
import warnings
import wave
from scipy import signal
import traceback  # Added for better error logging

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Modified HTML/JavaScript for real-time chunked recording
AUDIO_HTML = """
<script>
// Create UI elements
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var status_div = document.createElement("DIV");
status_div.style.marginTop = "10px";
status_div.innerHTML = "Ready to record";

my_btn.textContent = "Start Recording";
my_div.appendChild(my_btn);
my_div.appendChild(status_div);
document.body.appendChild(my_div);

// Variables for recording
var recorder, gumStream;
var recordButton = my_btn;
var isRecording = false;
var audioChunks = [];
var chunkInterval = 6000; // 3 seconds per chunk
var chunkTimer;
var transcriptionActive = false;

// Queue to store chunks
var chunkQueue = [];

// Function to handle successful media stream
function handleSuccess(stream) {
  gumStream = stream;
  var options = {
    mimeType: 'audio/webm;codecs=opus'
  };

  recorder = new MediaRecorder(stream, options);  // Added options parameter

  // Event triggered when data is available
  recorder.ondataavailable = function(e) {
    if (e.data.size > 0) {
      audioChunks.push(e.data);
      processChunk();
    }
  };

  // Start recording
  recorder.start();
  isRecording = true;
  recordButton.textContent = "Stop Recording";
  status_div.innerHTML = "Recording...";

  // Set up timer for chunk collection
  startChunkTimer();
}

// Timer function to collect audio in chunks
function startChunkTimer() {
  chunkTimer = setInterval(function() {
    if (recorder && recorder.state === "recording") {
      recorder.requestData(); // Request data without stopping recorder
    }
  }, chunkInterval);
}

// Process an audio chunk
function processChunk() {
  if (audioChunks.length > 0) {
    var latestChunk = audioChunks.pop();
    audioChunks = []; // Clear for next chunks

    // Create blob from the chunk
    var blob = new Blob([latestChunk], { type: 'audio/webm;codecs=opus' });

    // Convert to base64
    var reader = new FileReader();
    reader.onloadend = function() {
      var base64data = reader.result;
      // Add to queue for Python to process
      chunkQueue.push(base64data);
      status_div.innerHTML = `Recording... (${chunkQueue.length} chunks queued)`;
    };
    reader.readAsDataURL(blob);
  }
}

// Toggle recording on button click
recordButton.onclick = function() {
  if (!isRecording) {
    // Start recording
    transcriptionActive = true;
    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
  } else {
    // Stop recording
    clearInterval(chunkTimer);
    if (recorder) {
      recorder.stop();
      gumStream.getAudioTracks().forEach(track => track.stop());
    }
    isRecording = false;
    transcriptionActive = false;
    recordButton.textContent = "Start Recording";
    status_div.innerHTML = "Recording stopped";
  }
};

// Function to check if there are any chunks available
window.hasNextChunk = function() {
  return chunkQueue.length > 0;
};

// Function to get the next chunk
window.getNextChunk = function() {
  if (chunkQueue.length > 0) {
    return chunkQueue.shift();
  }
  return null;
};

// Function to check if transcription is active
window.isTranscriptionActive = function() {
  return transcriptionActive;
};
</script>
"""

# Improved function to convert base64 audio to audio data
def process_audio_chunk(base64data):
    """Process audio chunk using direct ffmpeg pipe to avoid file format issues"""
    try:
        # Verify base64data contains the expected format
        if not base64data or not isinstance(base64data, str) or ',' not in base64data:
            print("Invalid base64 data format")
            return None, None

        # Decode base64 data
        try:
            binary = b64decode(base64data.split(',')[1])
            if not binary:
                print("Empty binary data after decoding")
                return None, None
        except Exception as e:
            print(f"Base64 decoding error: {e}")
            return None, None

        # Use ffmpeg to convert to WAV
        try:
            # Set explicit audio parameters to ensure consistency
            process = (ffmpeg
                .input('pipe:0')
                .output('pipe:1', format='wav', acodec='pcm_s16le', ar='16000', ac='1')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True)
            )

            output, err = process.communicate(input=binary)

            if not output or len(output) < 100:  # Basic check for valid output
                print(f"ffmpeg produced insufficient output. Error: {err.decode() if err else 'None'}")
                return None, None

        except Exception as e:
            print(f"ffmpeg processing error: {e}")
            return None, None

        # Process the WAV data directly without using scipy.io.wavfile
        with io.BytesIO(output) as wav_io:
            try:
                # Open using wave library which is more robust for this purpose
                with wave.open(wav_io, 'rb') as wav:
                    # Get parameters
                    frames = wav.readframes(wav.getnframes())
                    rate = wav.getframerate()
                    channels = wav.getnchannels()
                    width = wav.getsampwidth()

                    # Validate the data
                    if len(frames) == 0:
                        print("No audio frames in the processed data")
                        return None, None

                    # Convert to numpy array
                    if width == 2:  # 16-bit audio
                        dtype = np.int16
                    elif width == 4:  # 32-bit audio
                        dtype = np.int32
                    else:
                        dtype = np.int8

                    audio = np.frombuffer(frames, dtype=dtype)

                    # If stereo, convert to mono by averaging channels
                    if channels == 2:
                        audio = audio.reshape(-1, 2).mean(axis=1).astype(dtype)

                    return audio, rate
            except Exception as e:
                print(f"Wave processing error: {e}")
                return None, None

    except Exception as e:
        print(f"Error processing audio chunk: {traceback.format_exc()}")
        # Return None instead of empty array to trigger alternate method
        return None, None

# Alternate method to process audio if traditional method fails
def process_audio_chunk_alternate(base64data):
    """Alternative method to process audio chunks if the primary method fails"""
    try:
        if not base64data or not isinstance(base64data, str) or ',' not in base64data:
            print("Invalid base64 data format in alternate method")
            return np.zeros(16000, dtype=np.int16), 16000

        # Decode base64 data
        binary = b64decode(base64data.split(',')[1])
        if len(binary) < 100:  # Check for meaningful content
            print("Too little data in binary content")
            return np.zeros(16000, dtype=np.int16), 16000

        # Save to a temporary file with a unique name to avoid conflicts
        temp_file = f'/tmp/temp_audio_{time.time()}.webm'
        temp_wav = f'/tmp/temp_audio_{time.time()}.wav'

        try:
            with open(temp_file, 'wb') as f:
                f.write(binary)

            # Use ffmpeg to convert to WAV file
            cmd = f'ffmpeg -i {temp_file} -ar 16000 -ac 1 -c:a pcm_s16le {temp_wav} -y -loglevel error'
            result = os.system(cmd)

            if result != 0:
                print(f"ffmpeg command failed with exit code {result}")
                return np.zeros(16000, dtype=np.int16), 16000

            # Read the WAV file
            if not os.path.exists(temp_wav) or os.path.getsize(temp_wav) < 100:
                print("Converted WAV file is missing or too small")
                return np.zeros(16000, dtype=np.int16), 16000

            with wave.open(temp_wav, 'rb') as wav:
                frames = wav.readframes(wav.getnframes())
                rate = wav.getframerate()
                if len(frames) == 0:
                    print("No frames in WAV file")
                    return np.zeros(16000, dtype=np.int16), 16000

                audio = np.frombuffer(frames, dtype=np.int16)

            return audio, rate

        finally:
            # Clean up
            if os.path.exists(temp_file):
                os.remove(temp_file)
            if os.path.exists(temp_wav):
                os.remove(temp_wav)

    except Exception as e:
        print(f"Error in alternate audio processing: {traceback.format_exc()}")
        return np.zeros(16000, dtype=np.int16), 16000

# Function to initialize the transcription model
def initialize_model(model_size="base"):
    print(f"Loading {model_size} Whisper model...")
    # Use either CPU or CUDA depending on availability
    model = WhisperModel(model_size, device="cuda" if "CUDA_VISIBLE_DEVICES" in os.environ else "cpu")
    print("Model loaded!")
    return model

# Chunked recording and real-time transcription
def record_chunk():
    """Get a single chunk of audio from the JavaScript interface"""
    # Check if a chunk is available
    has_chunk = eval_js("hasNextChunk()")

    if has_chunk:
        # Get the next audio chunk
        chunk_data = eval_js("getNextChunk()")
        if chunk_data:
            try:
                # Try primary method first
                audio, sr = process_audio_chunk(chunk_data)
                if audio is None or len(audio) == 0:
                    print("Primary method failed or produced empty audio, trying alternate method")
                    return process_audio_chunk_alternate(chunk_data)
                return audio, sr
            except Exception as e:
                print(f"Trying alternate method due to: {e}")
                # Try alternate method if primary fails
                return process_audio_chunk_alternate(chunk_data)

    return None, None

# Create a function to update transcription display
def update_transcription_display(text):
    from IPython.display import display as ipython_display
    from IPython.display import HTML as ipython_HTML
    # Add escaping for HTML special characters
    text = text.replace("<", "&lt;").replace(">", "&gt;")
    ipython_display(ipython_HTML(f"<script>document.getElementById('transcription-output').innerHTML = '<h3>Transcription:</h3>{text}';</script>"))

# Function to save audio data to a temporary file for transcription
def save_audio_for_transcription(audio, sr):
    """Save audio data to a temporary file in the correct format for transcription"""
    temp_file = io.BytesIO()

    # Validate input
    if audio is None or len(audio) == 0:
        print("Cannot save empty audio data")
        return None

    # Ensure audio is in the correct format
    if audio.dtype != np.int16:
        audio = audio.astype(np.int16)

    # Resample if necessary (whisper expects 16kHz)
    if sr != 16000:
        print(f"Resampling from {sr}Hz to 16000Hz")
        # Use a more sophisticated resampling approach
        try:
            resampled_len = int(len(audio) * 16000 / sr)
            audio = signal.resample(audio, resampled_len)
            sr = 16000
        except Exception as e:
            print(f"Resampling error: {e}")
            return None

    try:
        # Create WAV file
        with wave.open(temp_file, 'wb') as wav:
            wav.setnchannels(1)  # mono
            wav.setsampwidth(2)  # 16-bit
            wav.setframerate(sr)
            wav.writeframes(audio.tobytes())

        temp_file.seek(0)

        # Validate the file has actual content
        if temp_file.getbuffer().nbytes < 100:  # Arbitrary minimum size check
            print("Generated audio file is too small")
            return None

        return temp_file
    except Exception as e:
        print(f"Error saving audio: {e}")
        return None

# Main function for real-time transcription
def realtime_transcription(model_size="base", chunk_processing_queue=None):
    """
    Perform real-time transcription using Whisper model

    Args:
        model_size: Size of the Whisper model to use
        chunk_processing_queue: Queue to store processed chunks (optional)
    """
    # Display the recording interface
    display(HTML(AUDIO_HTML))

    print("Initializing transcription model...")
    model = initialize_model(model_size)

    # Create output area for transcriptions
    display(HTML("<div id='transcription-output' style='margin-top: 20px; padding: 10px; border: 1px solid #ccc; min-height: 100px;'><h3>Transcription:</h3></div>"))

    print("Ready! Click the button to start/stop recording.")
    print("Transcription will appear below as you speak.")

    # Initialize variables for transcription
    transcription_text = ""
    segment_history = []

    # Main loop
    try:
        while True:
            # Check if transcription is active
            is_active = eval_js("isTranscriptionActive()")

            if is_active:
                # Get a chunk of audio
                audio, sr = record_chunk()

                if audio is not None and len(audio) > 0:
                    # Ensure the audio has enough content to transcribe
                    if len(audio) < 1000:  # Very short audio, likely empty
                        print("Audio chunk too short, skipping")
                        continue

                    # Save audio to temporary file
                    temp_file = save_audio_for_transcription(audio, sr)
                    if temp_file is None:
                        print("Failed to save audio for transcription")
                        continue

                    try:
                        # Transcribe the chunk
                        segments, info = model.transcribe(temp_file, beam_size=5, language="en")

                        # Process segments
                        has_content = False
                        segment_texts = []

                        for segment in segments:
                            segment_text = segment.text.strip()
                            if segment_text:
                                segment_texts.append(segment_text)

                        if segment_texts:
                            has_content = True
                            current_segment = " ".join(segment_texts)
                            segment_history.append(current_segment)
                            print(f"Transcribed: {current_segment}")

                            # Keep only the last 10 segments to avoid too much text
                            if len(segment_history) > 10:
                                segment_history.pop(0)

                            # Update the transcription display
                            full_text = " ".join(segment_history)
                            update_transcription_display(full_text)

                            # Add to queue if provided
                            if chunk_processing_queue is not None:
                                chunk_processing_queue.put(current_segment)

                        if not has_content:
                            print("No transcription content in this chunk (possibly silence)")

                    except Exception as e:
                        print(f"Error during transcription: {traceback.format_exc()}")

            # Small delay to avoid hammering the browser
            time.sleep(0.1)

    except KeyboardInterrupt:
        print("Transcription stopped by user.")

    return segment_history

# Sample usage function for the entire system
def main():
    # Create a queue for processing transcribed chunks
    chunk_queue = queue.Queue()

    # Create a thread for any additional processing of transcribed text
    def process_transcription():
        while True:
            try:
                text = chunk_queue.get(timeout=1)
                # Do something with the transcribed text
                # For example, analyze sentiment, extract commands, etc.
                print(f"Processing: {text}")
                chunk_queue.task_done()
            except queue.Empty:
                time.sleep(3)
            except Exception as e:
                print(f"Error in processing thread: {traceback.format_exc()}")

    # Start processing in a separate thread
    processing_thread = threading.Thread(target=process_transcription)
    processing_thread.daemon = True
    processing_thread.start()

    # Start the real-time transcription
    try:
        return realtime_transcription(model_size="base", chunk_processing_queue=chunk_queue)
    except Exception as e:
        print(f"Error in transcription: {traceback.format_exc()}")
        return None

In [ ]:
# maybe adjust chunkInterval
transcription = main()

Initializing transcription model...
Loading base Whisper model...
Model loaded!


Ready! Click the button to start/stop recording.
Transcription will appear below as you speak.
Transcribed: Luke, I am your mother and father.


Processing: Luke, I am your mother and father.
ffmpeg produced insufficient output. Error: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-

In [ ]:
transcription

['Luke, I am your mother and father.']